# **PROYECTO CREW SCHEDULING** - David Pardo Peña

## **INICIALIZACIÓN :**

### **- Librerias y función lectura instancias**

In [1]:
#Importar librerias que se van a utilizar
import time
import math
import random
import pulp as pl
import numpy as np
import pandas as pd
import networkx as nx
from collections import deque
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.optimize import linear_sum_assignment


#Función lectura de instancias
def read_data(file_path):
    with open(file_path, 'r') as file:
        length = None
        num_employees = None
        num_shifts = None
        temporal_requirements = []
        shift_data = []
        min_days_off = None
        max_days_off = None
        min_work_blocks = None
        max_work_blocks = None
        num_sequences_of_length_2 = None
        num_sequences_of_length_3 = None
        forbidden_sequences = []

        for line in file:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            if length is None:
                length = int(line)
            elif num_employees is None:
                num_employees = int(line)
            elif num_shifts is None:
                num_shifts = int(line)
            elif len(temporal_requirements)<num_shifts:
                temporal_requirements.append(line.split())
            elif len(shift_data)<num_shifts:
                # Dividir la línea en palabras
                words = line.split()
                # Obtener el nombre del turno y los demás datos
                shift_name = words[0]
                start_time = int(words[1])
                lengthT = int(words[2])
                employee_requirements = list(map(int, words[3:]))
                # Agregar los datos a la lista de turnos
                shift_data.append({
                    "name": shift_name,
                    "start_time": start_time,
                    "lengthT": lengthT,
                    "employee_requirements": employee_requirements})
            elif min_days_off is None:
                min_days_off, max_days_off = map(int, line.split())
            elif min_work_blocks is None:
                min_work_blocks, max_work_blocks = map(int, line.split())
            elif num_sequences_of_length_2 is None:
                num_sequences_of_length_2, num_sequences_of_length_3 = map(int, line.split())
            else:
                forbidden_sequences.append(tuple(line.split()))
        data={
            "length": length,
            "num_employees": num_employees,
            "num_shifts": num_shifts,
            "temporal_requirements": temporal_requirements,
            "shift_data": shift_data,
            "min_days_off": min_days_off,
            "max_days_off": max_days_off,
            "min_work_blocks": min_work_blocks,
            "max_work_blocks": max_work_blocks,
            "num_sequences_of_length_2": num_sequences_of_length_2,
            "num_sequences_of_length_3": num_sequences_of_length_3,
            "forbidden_sequences": forbidden_sequences
             }
        
        return data


### **- Cargar Instancia**

In [2]:
##  CARGA DE DATOS Y PARAMETROS  ###############################################################################################################
import tkinter as tk
from tkinter import filedialog
import os

# Crear la ventana raíz oculta
root = tk.Tk()
root.withdraw()

# Obtener la ruta del directorio actual de trabajo
script_dir = os.getcwd()

# Abrir el cuadro de diálogo de selección de archivo con filtro para archivos de texto
archivo_seleccionado = filedialog.askopenfilename(
    initialdir=script_dir, 
    title="Selecciona un archivo de texto", 
    filetypes=[("Archivos de texto", "*.txt")]
)

# Verificar si se seleccionó un archivo y que sea un archivo de texto
if archivo_seleccionado:
    if archivo_seleccionado.endswith('.txt'):
        print(f"Archivo de texto seleccionado: {archivo_seleccionado}")
    else:
        print("El archivo seleccionado no es un archivo de texto.")
else:
    print("No se seleccionó ningún archivo.")


instancia=archivo_seleccionado
#instancia="integra_troncal.txt"
data = read_data(instancia)
print(data)

# Lista que guarda los tipos de turnos que hay en la instancia
tipo_turnos=[]
for i in range(len(data['shift_data'])):
    tipo_turnos.append(data['shift_data'][i]['name'])
    
# Diccionario que guarda el turno que que le gusta y no le gusta a cada conductor en una lista, la primera posición de la lista es el
# turno que le gusta y la segunda posición es el turno que no le gusta
preferencias = {}
if instancia != "integra_troncal.txt":
    generador = random.Random()
    generador.seed(9)
    for conductor in range(data['num_employees']):
        le_gusta = generador.choice(tipo_turnos)
        tipo_turnos1=tipo_turnos.copy()
        tipo_turnos1.remove(le_gusta)
        no_le_gusta = generador.choice(tipo_turnos1)
        preferencias[conductor] = [le_gusta,no_le_gusta]
else:
    preferencias = {0:('D','N'),1:('D','N'),2:('N','D'),3:('D','M'),4:('N','D'),5:('M','N'),6:('M','D'),7:('D','M'),8:('N','D'),9:('N','D'),10:('M','D'),11:('N','D'),12:('N','D'),13:('N','D'),14:('D','M'),15:('N','M'),16:('D','N'),17:('M','D'),18:('N','M'),19:('N','M'),20:('D','N'),21:('M','D'),22:('D','N'),23:('D','N'),24:('D','N'),25:('M','D'),26:('M','D'),27:('N','D'),28:('D','N'),29:('N','D'),30:('M','D'),31:('D','M'),32:('D','N'),33:('N','D'),34:('D','M'),35:('N','D'),36:('D','M'),37:('D','N'),38:('D','M'),39:('D','N'),40:('D','N'),41:('D','N'),42:('M','D'),43:('N','D'),44:('M','D'),45:('D','M'),46:('M','N'),47:('N','D'),48:('D','M'),49:('D','M'),50:('D','M'),51:('D','N'),52:('D','M'),53:('D','M'),54:('D','N'),55:('N','D'),56:('D','M'),57:('D','M'),58:('N','D'),59:('D','N'),60:('N','D'),61:('D','N'),62:('D','N'),63:('M','N'),64:('D','M'),65:('N','D'),66:('M','D'),67:('D','M'),68:('M','D'),69:('M','D'),70:('M','D'),71:('M','D'),72:('D','M'),73:('M','D'),74:('D','N'),75:('M','D'),76:('D','M'),77:('N','D'),78:('M','D'),79:('N','D'),80:('D','M'),81:('D','N')}

# Generar tabla de visualización de preferencias por conductor
tabla = pd.DataFrame(preferencias).transpose().rename(columns={0:'Les gusta',1:'No les gusta'})
tabla.reset_index(drop=True, inplace=True)
estilos = [    dict(selector="th", props=[("border", "1px solid black")]), dict(selector="td", props=[("border", "1px solid black")])]
tabla_estilizada = tabla.style.set_table_styles(estilos)
#display(HTML(tabla_estilizada.render()))

#Crea el diccionario 'turnos' que guarda los turnos (key = ID)
turnos = {}
for tipo in range(data['num_shifts']):
    for dia in range(data['length']):
        for requerimiento in range(int(data['temporal_requirements'][tipo][dia])):
            nombre=tipo_turnos[tipo]+"-dia "+str(dia)+"-"+str(requerimiento)
            turnos[tipo_turnos[tipo],dia,requerimiento]=nombre

# Maxima carga de trabajo (Turnos) que puede ser asignada a los conductores
maximo_final = min(data['length']-data['min_days_off'],data['max_work_blocks'])
# Lista que contiene los IDs de los conductores
conductores = list(range(data['num_employees']))
maximo_inicial = data['min_work_blocks']


##  VERIFICACIÓN  ###############################################################################################################


if instancia != "integra_troncal.txt":
    # Si la suma de todos los turnos que deben ser asignados es mayor a el numero de conductores por el maximo de turnos
    # que pueden ser asignados a cada conductor. Entonces calcula el numero de conductores que se tendrian que contratar
    # para cumplir con la asignación.
    Li=data['num_employees']
    L=data['num_employees']
    N=[]
    N.extend(data['temporal_requirements'])
    N2 = [int(x) for sublist in N for x in sublist]
    while sum(N2)>L*maximo_final:
        L+=1
    if L>Li:
        preferencias = {}
        for conductor in range(L):
            le_gusta = generador.choice(tipo_turnos)
            tipo_turnos1=tipo_turnos.copy()
            tipo_turnos1.remove(le_gusta)
            no_le_gusta = generador.choice(tipo_turnos1)
            preferencias[conductor] = [le_gusta,no_le_gusta]
        conductores = list(preferencias.keys())
        print("Se debe(n) contratar",L-Li,"conductor(es) para cumplir los requerimientos")

# Si la instancia esta mal diseñada puede pasar que al tener muchos conductores no se cumpl el minimo de horas que deben
# trabajar los conductores. Entonces calcula el numero de conductores que se tendrian que despedir para cumplir con esto.
tamaño = sum(int(data['temporal_requirements'][tipo][dia]) for tipo in range(data['num_shifts']) for dia in range(data['length']))
if data['min_work_blocks']*data['num_employees']>tamaño:
    valor = 0
    while data['min_work_blocks']*(data['num_employees']-valor)>tamaño:
        valor+=1    
    print('\n')
    print("Se debe(n) despedir",valor,"conductor(es) para cumplir los requerimientos de minimas horas de trabajo")

    for i in range(valor):
        conductores.remove(len(conductores)-1)

print(maximo_inicial,maximo_final)

Archivo de texto seleccionado: C:/Users/horse/OneDrive - Universidad de los andes/Documentos/TESIS/Instancias/Example20.txt
{'length': 7, 'num_employees': 163, 'num_shifts': 3, 'temporal_requirements': [['72', '79', '80', '78', '82', '76', '74'], ['39', '40', '44', '43', '43', '38', '40'], ['5', '6', '5', '6', '6', '6', '5']], 'shift_data': [{'name': 'D', 'start_time': 360, 'lengthT': 480, 'employee_requirements': [2, 6]}, {'name': 'A', 'start_time': 840, 'lengthT': 480, 'employee_requirements': [2, 6]}, {'name': 'N', 'start_time': 1320, 'lengthT': 480, 'employee_requirements': [2, 5]}], 'min_days_off': 1, 'max_days_off': 4, 'min_work_blocks': 3, 'max_work_blocks': 6, 'num_sequences_of_length_2': 3, 'num_sequences_of_length_3': 4, 'forbidden_sequences': [('N', 'D'), ('N', 'A'), ('A', 'D'), ('A', '-', 'D'), ('N', '-', 'A'), ('N', '-', 'D'), ('N', '-', 'N')]}
3 6


## **ALGORITMOS :**

### **- Funciones apoyo**: Función objetivo, cumplimiento restricciones y graficar solución

In [3]:
#Función para evaluar la satisfacción de una solución encontrada. Recibe como parametros el diccionario 'turnos' que para
#cada ID de turno almacena sus propiedades, el diccionario 'iconductor' que para cada ID de turno almacena el conductor asignado
#y el diccionario 'preferencias' que para cada conductor almacena una tupla con el turno que les gusta y el turno que no les gusta
def evaluar_satisfaccion(turnos,iconductor,preferencias): 
    #Inicializa el puntaje en 0
    puntaje = 0
    #Inicializa los contadores de satisfechos, indiferentes y insatisfechos
    satisfechos=0
    indiferentes=0
    insatisfechos=0
    #Itera sobre los IDs de turnos
    for t in list(turnos.keys()):
        #Obtiene el conductor signado al turno
        conduc = iconductor[t]
        #Revisa las preferencias del conductor y le da puntaje de 1 si le gusta, -1 si no le gusta y 0 si le es indiferente
        #también actualiza los contadores
        if preferencias[conduc][0]==turnos[t][0]:
            puntaje +=1
            satisfechos+=1
        elif preferencias[conduc][1]==turnos[t][0]:
            puntaje +=-1
            insatisfechos+=1
        else:
            indiferentes+=1
    #Retorna el puntaje obtenido y los contadores
    return puntaje,satisfechos,indiferentes,insatisfechos

def calcular_satisfaccion(asig): 
    puntaje = 0
    satisfechos=0
    indiferentes=0
    insatisfechos=0
    total = 0
    for tipo,dia,req in turnos:
        if preferencias[asig[tipo,dia,req][0]][0]==tipo:
            satisfechos+=1
            total+=1
        elif preferencias[asig[tipo,dia,req][0]][1]==tipo:
            insatisfechos+=1
            total+=1
        else:
            indiferentes+=1
            total+=1

    puntaje = satisfechos-insatisfechos/total
    
    return puntaje,satisfechos,indiferentes,insatisfechos

#Función para revisar que no se esté violando ninguna restricción del problema. Recibe como parametros el rostering (solución)
#la lista con los tipos de turnos, el maximo de carga de trabajo y la data de la instancia
def revisar_proh(M):
    #Contador del numero de errores encontados
    respuesta = 0
    #Recorre cada conductor del rostering
    for i in range(len(M)):
        #Crea el diccionario 'bloques' que nos dice el numeró de bloques de cada tipo de turno
        bloques = {k:1 for k in tipo_turnos}
        #Crea contador de carga de trabajo para el conductor
        carga = 0
        #Itera sobre los turnos asignados del conductor
        for j in range(len(M[i])):
            #Revisa las secuencias prohibidas de tres
            if j < len(M[i])-2:
                if (M[i][j],M[i][j+1],M[i][j+2]) in data['forbidden_sequences'] :
                    respuesta +=1
                    #print(f'\tError secuencia prohibida triple {(M[i][j],M[i][j+1],M[i][j+2])} conductor {i}')
            #Revisa las secuencias prohibidas de sencillas
            if j < len(M[i])-1:
                if (M[i][j],M[i][j+1]) in data['forbidden_sequences'] :
                    respuesta += 1
                    #print(f'\tError secuencia prohibida sencilla {(M[i][j],M[i][j+1])} conductor {i}')
            #Si el turno no es un turno de descanso
            if M[i][j] in tipo_turnos:
                #Incrementa la carga de trabajo del conductor
                carga += 1
                if j < len(M[i])-1: 
                    #Si el tipo de turno del dia siguiente es igual al tipo de turno del dia actual
                    if M[i][j]==M[i][j+1]:
                        #Actualiza los bloques para ese tipo de turno
                        bloques[M[i][j]]+=1
                        #Revisa el número de bloques para ese tipo de turno (Maximo de turnos que se pueden asignar de ese tipo a la semana)
                        if bloques[M[i][j]]>data['shift_data'][tipo_turnos.index(M[i][j])]['employee_requirements'][1]:
                            respuesta+=1
                            #print(f'\tError bloques {M[i][j]} en Conductor {i} -> {bloques[M[i][j]]}')
            else:
                for tipo in tipo_turnos:
                    bloques[tipo]=1
        #Revisa la restricción de maxima carga de trabajo de turnos por trabajador a la semana
        if carga > maximo_final:
            respuesta += 1
            #print(f'\tExcede carga en conductor {i} -> {carga}')

    return respuesta

#Función para generar las posiciones de los nodos del grafo para que quede organizado como una arbol de ancestros a predecesores
def generar_pos(G):

    roots = {n:(d,list(G.successors(n))) for n,d in G.in_degree()}
    t=len(roots)

    nivel={}
    while len(nivel)<len(roots):
        for n in roots:
            if roots[n][0]==0:
                nivel[n]=t
                for s in roots[n][1]:
                    nivel[s]=nivel[n]-1
            if n in list(nivel.keys()):
                l=nivel[n]
                for s in roots[n][1]:
                    nivel[s]=l-1
    minimo=min(nivel.values())               
    for n in nivel:
        nivel[n]-=minimo

    maximo=max(nivel.values())
    i=0
    nivelc={}
    while i<=maximo:
        cont=0
        for n in nivel:
            if nivel[n]==i:
                nivelc[n]=cont
                cont+=1
        i+=1

    pos={n:(nivelc[n],nivel[n]) for n in nivel}
    return pos


def secuencia_Valida(secuencia,revisar_carga = False):
    valida = True
    #Crea el diccionario 'bloques' que nos dice el numeró de bloques de cada tipo de turno
    bloques = {k:1 for k in tipo_turnos}
    #Variable que guarda la carga de la secuencia
    carga = 0
    #Itera sobre los turnos asignados del conductor
    for i in range(len(secuencia)):
        #Condicion para actualizar carga
        if secuencia[i]!='-':
            carga+=1
        #Revisa las secuencias prohibidas de tres
        if i < len(secuencia)-2:
            if (secuencia[i],secuencia[i+1],secuencia[i+2]) in data['forbidden_sequences'] :
                valida = False
                break
        #Revisa las secuencias prohibidas de sencillas
        if i < len(secuencia)-1:
            if (secuencia[i],secuencia[i+1]) in data['forbidden_sequences'] :
                valida = False
                break
        #Si el turno no es un turno de descanso
        if secuencia[i] in tipo_turnos:
            if i < len(secuencia)-1: 
                #Si el tipo de turno del dia siguiente es igual al tipo de turno del dia actual
                if secuencia[i]==secuencia[i+1]:
                    #Actualiza los bloques para ese tipo de turno
                    bloques[secuencia[i]]+=1
                    #Revisa el número de bloques para ese tipo de turno (Maximo de turnos que se pueden asignar de ese tipo a la semana)
                    if bloques[secuencia[i]]>data['shift_data'][tipo_turnos.index(secuencia[i])]['employee_requirements'][1]:
                        valida = False
                        break
        else:
            for tipo in tipo_turnos:
                bloques[tipo]=1

    #En caso de que este activa la varibale para revisar la carga
    if revisar_carga:
        if carga>maximo_final or carga<maximo_inicial:
            valida = False

    return valida

#Función que nos muestra visualmente la solución encontrada
def graficar(max_v,max_t,r0,r1,r2,r3,D,solucion2,turnos2,rostering2,factible,fin,puntaje_inicio,puntaje_Split,rostering_Split,tiempo_Split):
    M=[]
    M.append(r0)
    M.append(r3)
    M.append(r1)
    M.append(r2)
    tabla = pd.DataFrame(M).transpose().rename(columns={0:'Exitosas',1:'No exitosas',2:'Reparadas',3:'Reparación fallida'})
    tabla.reset_index(drop=True, inplace=True)
    estilos = [    dict(selector="th", props=[("border", "1px solid black")]),
        dict(selector="td", props=[("border", "1px solid black")])
    ]
    tabla = tabla.rename(index={0: '#'})
    tabla_estilizada = tabla.style.set_table_styles(estilos)
    display(tabla_estilizada)

    print(f"Tiempo de ejecución total -> {D+fin+tiempo_Split} s")
    
    if r0>0:
        #Solución GRASP
        for j in max_t[1][0]:
            if max_t[1][0][j]==-1:
                "falta turnos por asignar"
        asignacion = {}
        for turno in max_t[1][0]:
            asignacion[turno]=(max_t[1][0][turno],turno)
        v=calcular_satisfaccion(asignacion)
        puntaje = v[0]
        satisfechos=v[1]
        indiferentes=v[2]
        insatisfechos=v[3]
        M=[]
        M.append(str(D))
        M.append(str(round(max_t[2],2)))
        M.append(str(round(puntaje,2)))
        M.append(str(round(satisfechos/max_t[0][2]*100,2))+' %')
        M.append(str(round(indiferentes/max_t[0][2]*100,2))+' %')
        M.append(str(round(insatisfechos/max_t[0][2]*100,2))+' %')
        tabla = pd.DataFrame(M).transpose().rename(columns={0:'Tiempo (s)',1:'Puntaje Constructivo',2:'Puntaje Busqueda Local',3:'Satisfechos',4:'Indiferentes',5:'Insatisfechos'})
        tabla.reset_index(drop=True, inplace=True)
        estilos = [    dict(selector="th", props=[("border", "1px solid black")]),
            dict(selector="td", props=[("border", "1px solid black")])
        ]
        tabla = tabla.rename(index={0: 'Estadisticas'})
        tabla_estilizada = tabla.style.set_table_styles(estilos)
        display(tabla_estilizada)
        nuevo_puntaje_GRASP = satisfechos - insatisfechos/max_t[0][2]
        #Solución Constructivo asignación
        cont=0
        satisfechos=0
        indiferentes=0
        insatisfechos=0
        for c in conductores:
            for turno in solucion2[c]:
                if preferencias[c][0]==turnos2[turno][0]:
                    satisfechos+=1
                elif preferencias[c][1]==turnos2[turno][0]:
                    insatisfechos+=1
                else:
                    indiferentes+=1
                cont+=1
        total = satisfechos+indiferentes+insatisfechos
        nuevo_puntaje_constructivo = satisfechos - insatisfechos/total
        M=[]
        M.append(str(fin))
        M.append(str(round(nuevo_puntaje_constructivo,2)))
        M.append(str(round(satisfechos/total*100,2))+' %')
        M.append(str(round(indiferentes/total*100,2))+' %')
        M.append(str(round(insatisfechos/total*100,2))+' %')
        M.append(str(factible))
        tabla = pd.DataFrame(M).transpose().rename(columns={0:'Tiempo (s)',1:'Puntaje Constructivo Asignación',2:'Satisfechos',3:'Indiferentes',4:'Insatisfechos',5:'¿Factible?'})
        tabla.reset_index(drop=True, inplace=True)
        estilos = [    dict(selector="th", props=[("border", "1px solid black")]),
            dict(selector="td", props=[("border", "1px solid black")])
        ]
        tabla = tabla.rename(index={0: 'Estadisticas'})
        tabla_estilizada = tabla.style.set_table_styles(estilos)
        display(tabla_estilizada)
        #Solución SPLIT
        solucion_Split = [[turno for turno in rostering_Split[i] if turno!='-'] for i in range(len(rostering_Split))]
        cont=0
        satisfechos=0
        indiferentes=0
        insatisfechos=0
        for c in conductores:
            for turno in solucion_Split[c]:
                if preferencias[c][0]==turno:
                    satisfechos+=1
                elif preferencias[c][1]==turno:
                    insatisfechos+=1
                else:
                    indiferentes+=1
                cont+=1

        total = satisfechos+indiferentes+insatisfechos

        M=[]
        M.append(str(tiempo_Split))
        M.append(str(round(satisfechos-insatisfechos/total,2)))
        M.append(str(round(satisfechos/total*100,2))+' %')
        M.append(str(round(indiferentes/total*100,2))+' %')
        M.append(str(round(insatisfechos/total*100,2))+' %')
        M.append(str(len(conductores)))
        M.append(str(len(rostering_Split)))
        tabla = pd.DataFrame(M).transpose().rename(columns={0:'Tiempo (s)',1:'Puntaje SPLIT',2:'Satisfechos',3:'Indiferentes',4:'Insatisfechos',5:'#Conductores',6:'#Conductores solución'})
        tabla.reset_index(drop=True, inplace=True)
        estilos = [    dict(selector="th", props=[("border", "1px solid black")]),
            dict(selector="td", props=[("border", "1px solid black")])
        ]
        tabla = tabla.rename(index={0: 'Estadisticas'})
        tabla_estilizada = tabla.style.set_table_styles(estilos)
        display(tabla_estilizada)
        #Selección mejor solución
        if nuevo_puntaje_GRASP>nuevo_puntaje_constructivo and ((nuevo_puntaje_GRASP>puntaje_Split and len(rostering_Split)==len(conductores))or len(rostering_Split)>len(conductores)):
            M=[]
            for i in conductores:
                c=["-" for i in range(data['length'])]
                for d1 in range(data['length']):
                    for t,d,r in max_t[1][0]:
                        if max_t[1][0][t,d,r]==i and d1==d:
                            if c[d1]!="-":
                                print("********************ERROR********************")
                            c[d1]=t
                            break
                M.append(c)
            print("Mejor solución GRASP:")
            print("----------------------------------------------")
            print(revisar_proh(M),"Errores de secuencias prohibidas")
            print(sum(max_t[0][1]),"de",max_t[0][2],"turnos asignados")
            print("# Dias de trabajo  (min y max):",min(max_t[0][1]),"-",max(max_t[0][1]),"(",data['min_work_blocks'],"-",data['max_work_blocks'],")")
            print("# Dias de descanso (min y max):",data['length']-max(max_t[0][1]),"-",data['length']-min(max_t[0][1]),"(",data['min_days_off'],"-",data['max_days_off'],")")
            print("----------------------------------------------")
            

            tabla = pd.DataFrame(M,columns=["L","M", "X", "J", "V", "S", "D"])
            tabla_con_indices = tabla.set_index(pd.Index(["EMPLEADO "+str(i+1) for i in conductores]))
            estilos = [    dict(selector="th", props=[("border", "1px solid black")]),
                dict(selector="td", props=[("border", "1px solid black")])
            ]
            tabla_estilizada = tabla_con_indices.style.set_table_styles(estilos)

            display(tabla_estilizada)
        else:
            if factible and (len(rostering2)<len(rostering_Split)or (len(rostering2)==len(rostering_Split) and puntaje_Split>=puntaje_inicio )):
                print("Mejor solución asignación secuencial:")
                print("----------------------------------------------")
                print(revisar_proh(rostering2),"Errores de secuencias prohibidas")
                print(total,"de",cont,"turnos asignados")
                min_trabajado = min(len(secuencia) for secuencia in solucion2)
                max_trabajado = max(len(secuencia) for secuencia in solucion2)
                print("# Dias de trabajo  (min y max):",min_trabajado,"-",max_trabajado,"(",data['min_work_blocks'],"-",data['max_work_blocks'],")")
                print("# Dias de descanso (min y max):",data['length']-max_trabajado,"-",data['length']-min_trabajado,"(",data['min_days_off'],"-",data['max_days_off'],")")
                print("----------------------------------------------")

                tabla = pd.DataFrame(rostering2,columns=["L","M", "X", "J", "V", "S", "D"])
                tabla_con_indices = tabla.set_index(pd.Index(["EMPLEADO "+str(i+1) for i in conductores]))
                estilos = [    dict(selector="th", props=[("border", "1px solid black")]),
                    dict(selector="td", props=[("border", "1px solid black")])
                ]
                tabla_estilizada = tabla_con_indices.style.set_table_styles(estilos)

                display(tabla_estilizada)
            else:
                print("Mejor solución SPLIT:")
                print("----------------------------------------------")
                print(revisar_proh(rostering_Split),"Errores de secuencias prohibidas")
                print(total,"de",cont,"turnos asignados")
                min_trabajado = min(len(secuencia) for secuencia in solucion_Split)
                max_trabajado = max(len(secuencia) for secuencia in solucion_Split)
                print("# Dias de trabajo  (min y max):",min_trabajado,"-",max_trabajado,"(",data['min_work_blocks'],"-",data['max_work_blocks'],")")
                print("# Dias de descanso (min y max):",data['length']-max_trabajado,"-",data['length']-min_trabajado,"(",data['min_days_off'],"-",data['max_days_off'],")")
                print("----------------------------------------------")

                tabla = pd.DataFrame(rostering_Split,columns=["L","M", "X", "J", "V", "S", "D"])
                tabla_con_indices = tabla.set_index(pd.Index(["EMPLEADO "+str(i+1) for i in range(len(rostering_Split))]))
                estilos = [    dict(selector="th", props=[("border", "1px solid black")]),
                    dict(selector="td", props=[("border", "1px solid black")])
                ]
                tabla_estilizada = tabla_con_indices.style.set_table_styles(estilos)

                display(tabla_estilizada)
    
    elif factible and (len(rostering2)<len(rostering_Split)or (len(rostering2)==len(rostering_Split) and puntaje_Split==puntaje_inicio )):
        #Solución Constructivo asignación
        cont=0
        satisfechos=0
        indiferentes=0
        insatisfechos=0
        for c in conductores:
            for turno in solucion2[c]:
                if preferencias[c][0]==turnos2[turno][0]:
                    satisfechos+=1
                elif preferencias[c][1]==turnos2[turno][0]:
                    insatisfechos+=1
                else:
                    indiferentes+=1
                cont+=1
        total = satisfechos+indiferentes+insatisfechos
        nuevo_puntaje_constructivo = satisfechos - insatisfechos/total
        M=[]
        M.append(str(fin))
        M.append(str(round(nuevo_puntaje_constructivo,2)))
        M.append(str(round(satisfechos/total*100,2))+' %')
        M.append(str(round(indiferentes/total*100,2))+' %')
        M.append(str(round(insatisfechos/total*100,2))+' %')
        M.append(str(factible))
        tabla = pd.DataFrame(M).transpose().rename(columns={0:'Tiempo (s)',1:'Puntaje Constructivo Asignación',2:'Satisfechos',3:'Indiferentes',4:'Insatisfechos',5:'¿Factible?'})
        tabla.reset_index(drop=True, inplace=True)
        estilos = [    dict(selector="th", props=[("border", "1px solid black")]),
            dict(selector="td", props=[("border", "1px solid black")])
        ]
        tabla = tabla.rename(index={0: 'Estadisticas'})
        tabla_estilizada = tabla.style.set_table_styles(estilos)
        display(tabla_estilizada)
        #Solución SPLIT
        solucion_Split = [[turno for turno in rostering_Split[i] if turno!='-'] for i in range(len(rostering_Split))]
        cont=0
        satisfechos=0
        indiferentes=0
        insatisfechos=0
        for c in conductores:
            for turno in solucion_Split[c]:
                if preferencias[c][0]==turno:
                    satisfechos+=1
                elif preferencias[c][1]==turno:
                    insatisfechos+=1
                else:
                    indiferentes+=1
                cont+=1

        total = satisfechos+indiferentes+insatisfechos

        M=[]
        M.append(str(tiempo_Split))
        M.append(str(round(satisfechos-insatisfechos/total,2)))
        M.append(str(round(satisfechos/total*100,2))+' %')
        M.append(str(round(indiferentes/total*100,2))+' %')
        M.append(str(round(insatisfechos/total*100,2))+' %')
        M.append(str(len(conductores)))
        M.append(str(len(rostering_Split)))
        tabla = pd.DataFrame(M).transpose().rename(columns={0:'Tiempo (s)',1:'Puntaje SPLIT',2:'Satisfechos',3:'Indiferentes',4:'Insatisfechos',5:'#Conductores',6:'#Conductores solución'})
        tabla.reset_index(drop=True, inplace=True)
        estilos = [    dict(selector="th", props=[("border", "1px solid black")]),
            dict(selector="td", props=[("border", "1px solid black")])
        ]
        tabla = tabla.rename(index={0: 'Estadisticas'})
        tabla_estilizada = tabla.style.set_table_styles(estilos)
        display(tabla_estilizada)

        print("Mejor solución asignación secuencial:")
        print("----------------------------------------------")
        print(revisar_proh(rostering2),"Errores de secuencias prohibidas")
        print(total,"de",cont,"turnos asignados")
        min_trabajado = min(len(secuencia) for secuencia in solucion2)
        max_trabajado = max(len(secuencia) for secuencia in solucion2)
        print("# Dias de trabajo  (min y max):",min_trabajado,"-",max_trabajado,"(",data['min_work_blocks'],"-",data['max_work_blocks'],")")
        print("# Dias de descanso (min y max):",data['length']-max_trabajado,"-",data['length']-min_trabajado,"(",data['min_days_off'],"-",data['max_days_off'],")")
        print("----------------------------------------------")

        tabla = pd.DataFrame(rostering2,columns=["L","M", "X", "J", "V", "S", "D"])
        tabla_con_indices = tabla.set_index(pd.Index(["EMPLEADO "+str(i+1) for i in conductores]))
        estilos = [    dict(selector="th", props=[("border", "1px solid black")]),
            dict(selector="td", props=[("border", "1px solid black")])
        ]
        tabla_estilizada = tabla_con_indices.style.set_table_styles(estilos)

        display(tabla_estilizada)
    else:
        #Solución Constructivo asignación
        cont=0
        satisfechos=0
        indiferentes=0
        insatisfechos=0
        for c in conductores:
            for turno in solucion2[c]:
                if preferencias[c][0]==turnos2[turno][0]:
                    satisfechos+=1
                elif preferencias[c][1]==turnos2[turno][0]:
                    insatisfechos+=1
                else:
                    indiferentes+=1
                cont+=1
        total = satisfechos+indiferentes+insatisfechos
        nuevo_puntaje_constructivo = satisfechos - insatisfechos/total
        M=[]
        M.append(str(fin))
        M.append(str(round(nuevo_puntaje_constructivo,2)))
        M.append(str(round(satisfechos/total*100,2))+' %')
        M.append(str(round(indiferentes/total*100,2))+' %')
        M.append(str(round(insatisfechos/total*100,2))+' %')
        M.append(str(factible))
        tabla = pd.DataFrame(M).transpose().rename(columns={0:'Tiempo (s)',1:'Puntaje Constructivo Asignación',2:'Satisfechos',3:'Indiferentes',4:'Insatisfechos',5:'¿Factible?'})
        tabla.reset_index(drop=True, inplace=True)
        estilos = [    dict(selector="th", props=[("border", "1px solid black")]),
            dict(selector="td", props=[("border", "1px solid black")])
        ]
        tabla = tabla.rename(index={0: 'Estadisticas'})
        tabla_estilizada = tabla.style.set_table_styles(estilos)
        display(tabla_estilizada)
        #Solución SPLIT
        solucion_Split = [[turno for turno in rostering_Split[i] if turno!='-'] for i in range(len(rostering_Split))]
        cont=0
        satisfechos=0
        indiferentes=0
        insatisfechos=0
        for c in conductores:
            for turno in solucion_Split[c]:
                if preferencias[c][0]==turno:
                    satisfechos+=1
                elif preferencias[c][1]==turno:
                    insatisfechos+=1
                else:
                    indiferentes+=1
                cont+=1

        total = satisfechos+indiferentes+insatisfechos

        M=[]
        M.append(str(tiempo_Split))
        M.append(str(round(satisfechos-insatisfechos/total,2)))
        M.append(str(round(satisfechos/total*100,2))+' %')
        M.append(str(round(indiferentes/total*100,2))+' %')
        M.append(str(round(insatisfechos/total*100,2))+' %')
        M.append(str(len(conductores)))
        M.append(str(len(rostering_Split)))
        tabla = pd.DataFrame(M).transpose().rename(columns={0:'Tiempo (s)',1:'Puntaje SPLIT',2:'Satisfechos',3:'Indiferentes',4:'Insatisfechos',5:'#Conductores',6:'#Conductores solución'})
        tabla.reset_index(drop=True, inplace=True)
        estilos = [    dict(selector="th", props=[("border", "1px solid black")]),
            dict(selector="td", props=[("border", "1px solid black")])
        ]
        tabla = tabla.rename(index={0: 'Estadisticas'})
        tabla_estilizada = tabla.style.set_table_styles(estilos)
        display(tabla_estilizada)

        print("Mejor solución SPLIT:")
        print("----------------------------------------------")
        print(revisar_proh(rostering_Split),"Errores de secuencias prohibidas")
        print(total,"de",cont,"turnos asignados")
        min_trabajado = min(len(secuencia) for secuencia in solucion_Split)
        max_trabajado = max(len(secuencia) for secuencia in solucion_Split)
        print("# Dias de trabajo  (min y max):",min_trabajado,"-",max_trabajado,"(",data['min_work_blocks'],"-",data['max_work_blocks'],")")
        print("# Dias de descanso (min y max):",data['length']-max_trabajado,"-",data['length']-min_trabajado,"(",data['min_days_off'],"-",data['max_days_off'],")")
        print("----------------------------------------------")

        tabla = pd.DataFrame(rostering_Split,columns=["L","M", "X", "J", "V", "S", "D"])
        tabla_con_indices = tabla.set_index(pd.Index(["EMPLEADO "+str(i+1) for i in range(len(rostering_Split))]))
        estilos = [    dict(selector="th", props=[("border", "1px solid black")]),
            dict(selector="td", props=[("border", "1px solid black")])
        ]
        tabla_estilizada = tabla_con_indices.style.set_table_styles(estilos)

        display(tabla_estilizada)

        


        



### **- Algoritmo Asignación secuencial**

In [4]:
def Constructivo(p,q,proh,itera,acum):
    #Empieza a contabilizar el tiempo de ejecución
    inicio=time.time()
    #Inicializa la variable 'graficar', dependiendo de si quiero graficar la solución al final
    graficar = True
    #Crea el diccionario 'turnos' que guarda los turnos (key = ID), como no ha empezado el algoritmo crea
    #turnos ficticios que no contienen tipo y estan en el dia -1
    turnos = {i:("-",-1) for i in conductores}
    #Crea la lista 'iturnos' que contiene el ID de los turnos del dia donde estamos parados actualmente
    # (Dia ficticio -1) - Es un turno que no es de ningún tipo
    iturnos = conductores.copy()
    #Crea Diccionario 'iconductor' que dice a que ID (Turno) ha sido asignado el conductor en el dia actual (-1)
    iconductor = {i:i for i in conductores}
    #Crea el diccionario de 'cargaTrabajo' que guarda el numero de turnos que se le han asignado 
    #a cada conductor
    cargaTrabajo = {i:0 for i in conductores}
    #Crea el diccionario de 'bloquesTrabajo' que guarda el numero de turnos del mismo tipo que han
    #sido asignados a cada conductor de forma consecutiva
    bloquesTrabajos = {(i,j):1 for i in conductores for j in tipo_turnos}
    #Crea el diccionario 'prec' que almacena la precedencia del turno segun su ID
    prec = {}
    #Lista que guarda los arcos definitivos de la solución para visualizar la solución
    B = []
    #Diccionario que almacena el rostering (cronograma de asignación de turnos)
    M = {}
    #Crea el diccionario 'indexRow' que nos dice en que posición esta cada turno de 'iturnos'
    indexRow=iconductor.copy()
    #Diccionario 
    Errores = {}
    #Crea la lista 'Respuesta' donde cada posicion es el conductor y contiene una lista con los turnos asignados
    Respuesta = [[] for i in conductores]
    #Itera en los dias del horizonte de planeación (En cada iteración nos paramos en el dia j)
    for dia in range(data['length']):
        #Crea la lista 'jturnos' que contiene el ID de los turnos del dia j
        jturnos = []
        #Crea la lista 'Arcos' para guardar las tuplas de arcos disponibles para conectar los turnos en 
        #i(dia anterior) y j (dia actual)
        Arcos = []
        #Crea un diccionario que almacena los costos para resolver el problema de asignación dados los turnos del dia
        #anterior y los turnos disponibles para el dia en el que iteramos
        c = {}
        cost_matrix=[[1e9 for i in range(len(conductores))]  for i in iturnos]
        #Crea el diccionario 'indexCol' que nos dice en que posición esta cada turno de 'jturnos'
        indexCol = {}
        #Contador para saber en que posicion esta un turno de 'jturnos' en la 'cost_matrix'
        columnCont=0
        #Calcula el numero de turnos de descansos en el dia j 
        nDescansos = len(conductores)-sum(int(data['temporal_requirements'][tipo][dia]) for tipo in range(data['num_shifts']))
        #Recorre los turnos del dia anterior (dia i)
        for i in iturnos:
            #Guarda en la variable 'conduc' el conductor asignado a ese turno del dia anterior
            conduc = iconductor[i]
            #Recorre todos los tipos de turnos
            for tipo in range(data['num_shifts']):
                #Recorre sobre el requerimiento de turnos de ese tipo de turno en el dia actual
                for requerimiento in range(int(data['temporal_requirements'][tipo][dia])):
                    #El nombre del turno sera el ID del nuevo turno que se añadira
                    nombre=tipo_turnos[tipo]+"-dia "+str(dia)+"-"+str(requerimiento)
                    #Agrega al diccionario 'turnos' solamente en la revisión del ultimo turno del dia anterior
                    #Para evitar hacerlo varias veces teniendo en cuenta los ciclos internos
                    if i==iturnos[0]:
                        #Crea el turno y lo agrega al diccionario que tiene todos los turnos
                        turnos[nombre]=(tipo_turnos[tipo],dia,requerimiento)
                        #Agrega el ID del nuevo turno a la lista 'jturnos' - Turnos que deben ser asignados
                        jturnos.append(nombre)
                        #Agregamos la posicion en la que se encuentra el turno en 'jturnos'
                        indexCol[nombre]=columnCont
                        #El contador de columnas de turnos del dia j aumenta
                        columnCont+=1
                    #Crea variable para revisar si hay alguna violación de secuencia triple
                    secuencia_triple = 0
                    #Revisa que no haya una secuencia prohibida triple siempre y cuando ya estemos en el segundo dia en adelante
                    if dia>=2:
                        if (turnos[prec[i]][0],turnos[i][0],tipo_turnos[tipo]) in data['forbidden_sequences']:
                            secuencia_triple = True
                    #Revisa que el turno anterior no sea un turno ficticio
                    if turnos[i][0] in tipo_turnos:
                        #Revisa que se cumpla la restricción de carga de trabajo para el conductor y que no haya una secuencia
                        #prohibida sencilla, para poder generar la conexión entre ambos arcos
                        if (turnos[i][0],tipo_turnos[tipo]) not in data['forbidden_sequences'] and secuencia_triple == False and cargaTrabajo[iconductor[i]]<maximo_final and (i,nombre) not in proh:
                            #Si el turno anterior es del mismo tipo al actual entonces revisa la restricción de bloques de trabajo para ese tipo de turno
                            if turnos[i][0]==tipo_turnos[tipo] and bloquesTrabajos[conduc,tipo_turnos[tipo]]<data['shift_data'][tipo]['employee_requirements'][1]:
                                #Añade el arco - conecta ambos turnos porque es factible que el conductor haga ese turno
                                Arcos.append((i,nombre))
                                #Revisa las preferencias del conductor para darle un peso al arco. Como vamos a resolver un problema
                                #asignación debe darle un menor costo al turno con el que se sienta más a gusto.
                                #(0=le gusta,1=indiferencia,2=no le gusta)
                                #Adicionalmente hay una penalización p dependiendo de que tanta carga se le haya dado al conductor y
                                #una penalización q si ya tiene varios bloques de trabajo seguido ese conductor.
                                if preferencias[conduc][0]==tipo_turnos[tipo]:
                                    c[i,nombre]=0+cargaTrabajo[conduc]*p+bloquesTrabajos[iconductor[i],tipo_turnos[tipo]]*q
                                    cost_matrix[indexRow[i]][indexCol[nombre]] = 0+cargaTrabajo[conduc]*p+bloquesTrabajos[iconductor[i],tipo_turnos[tipo]]*q
                                elif preferencias[conduc][1]==tipo_turnos[tipo]:
                                    c[i,nombre]=2+cargaTrabajo[conduc]*p+bloquesTrabajos[iconductor[i],tipo_turnos[tipo]]*q
                                    cost_matrix[indexRow[i]][indexCol[nombre]] = 2+cargaTrabajo[conduc]*p+bloquesTrabajos[iconductor[i],tipo_turnos[tipo]]*q
                                else:
                                    c[i,nombre]=1+cargaTrabajo[conduc]*p+bloquesTrabajos[iconductor[i],tipo_turnos[tipo]]*q
                                    cost_matrix[indexRow[i]][indexCol[nombre]] = 1+cargaTrabajo[conduc]*p+bloquesTrabajos[iconductor[i],tipo_turnos[tipo]]*q
                            #Si los arcos no son del mismo tipo no revisa la restricción de bloques de trabajo por turno. Luego hace lo mismo a lo de arriba
                            elif turnos[i][0]!=tipo_turnos[tipo]:
                                Arcos.append((i,nombre))
                                if preferencias[conduc][0]==tipo_turnos[tipo]:
                                    c[i,nombre]=0+cargaTrabajo[conduc]*p+bloquesTrabajos[iconductor[i],tipo_turnos[tipo]]*q
                                    cost_matrix[indexRow[i]][indexCol[nombre]] = 0+cargaTrabajo[conduc]*p+bloquesTrabajos[iconductor[i],tipo_turnos[tipo]]*q
                                elif preferencias[conduc][1]==tipo_turnos[tipo]:
                                    c[i,nombre]=2+cargaTrabajo[conduc]*p+bloquesTrabajos[iconductor[i],tipo_turnos[tipo]]*q
                                    cost_matrix[indexRow[i]][indexCol[nombre]] = 2+cargaTrabajo[conduc]*p+bloquesTrabajos[iconductor[i],tipo_turnos[tipo]]*q
                                else:
                                    c[i,nombre]=1+cargaTrabajo[conduc]*p+bloquesTrabajos[iconductor[i],tipo_turnos[tipo]]*q
                                    cost_matrix[indexRow[i]][indexCol[nombre]] = 1+cargaTrabajo[conduc]*p+bloquesTrabajos[iconductor[i],tipo_turnos[tipo]]*q
                    #En caso de que el turno anterior sea ficcticio no revisa ninguna violación y hace directamente el calculo de la matriz de costos para la asignación
                    else:
                        if (turnos[i][0],tipo_turnos[tipo]) not in data['forbidden_sequences'] and secuencia_triple == False and cargaTrabajo[iconductor[i]]<maximo_final and (i,nombre) not in proh:
                            Arcos.append((i,nombre))
                            if preferencias[conduc][0]==tipo_turnos[tipo]:
                                c[i,nombre]=0+cargaTrabajo[conduc]*p
                                cost_matrix[indexRow[i]][indexCol[nombre]]=0+cargaTrabajo[conduc]*p
                            elif preferencias[conduc][1]==tipo_turnos[tipo]:
                                c[i,nombre]=2+cargaTrabajo[conduc]*p
                                cost_matrix[indexRow[i]][indexCol[nombre]]=2+cargaTrabajo[conduc]*p
                            else:
                                c[i,nombre]=1+cargaTrabajo[conduc]*p
                                cost_matrix[indexRow[i]][indexCol[nombre]]=1+cargaTrabajo[conduc]*p

            #Añade los turnos de descanso el diccionario 'turnos' y a la lista 'jturnos' con su respectivo ID
            #adicionalmente permite que todos los turnos anteriores puedan llegar al turno de descanso
            #tambien añade a la matriz de costos para la asignación un costo de 1, teniendo en cuenta que es mejor 
            #que ser asignado que a un turno con indiferencia
            for reqDescanso in range(nDescansos):
                nombre="Descanso"+"-dia "+str(dia)+"-"+str(reqDescanso)
                if i==iturnos[0]:
                    turnos[nombre]=("-",dia,1)
                    jturnos.append(nombre)
                    indexCol[nombre]=columnCont
                    columnCont+=1
                Arcos.append((i,nombre))
                c[i,nombre]=1
                cost_matrix[indexRow[i]][indexCol[nombre]]=1

        #RESOLVER ASIGNACIÓN PARA EL DIA ACTUAL

        for (i,j) in proh:
            if i in turnos:
                if dia == turnos[i][1]+1:
                    cost_matrix[indexRow[i]][indexCol[j]] = np.inf

        inicio_asig=time.time()
        #Variable que me dice si es factible la asignación o necesita ajustar asignaciones anteriores
        recalcular=False

        #Aplicamos metodo hungaro
        try:
            row_ind, col_ind = linear_sum_assignment(np.array(cost_matrix))

            for i_id, j_id in zip(row_ind, col_ind):
                i = iturnos[i_id]
                j = jturnos[j_id]
                if (i,j) not in Arcos:
                    if itera>0:
                        recalcular = True
                if turnos[j][0]!="-":
                    Respuesta[iconductor[i]].append(j)

        except ValueError:
            recalcular = True
                
        #En dado caso de que no debamos recalcular
        if recalcular == False:
            #Leemos solución obtenida con metodo hungaro
            for i_id, j_id in zip(row_ind, col_ind):
                    i = iturnos[i_id]
                    j = jturnos[j_id]
                    #Actualizamos la lista de arcos activos en la solución final
                    B.append((i,j))
                    if (i,j) not in Arcos:
                        Errores[iconductor[i]] = (turnos[prec[i]][1],turnos[i][1],turnos[j][1])
            
                    #Actualizamos el turno en el que se encuentra cada conductor
                    iconductor[j]=iconductor[i]
                    #Añadimos la precedencia dada la asignación del turno que acabamos de hacer
                    prec[j]=i
                    #Revisamos que el turno actual no sea ficticio
                    if turnos[j][0] in tipo_turnos:
                        #Actualizamos el diccionario de 'cargaTabajo'
                        cargaTrabajo[iconductor[j]]=cargaTrabajo[iconductor[j]]+1
                        #Actualizamos el diccionario de 'bloqueTabajo' 
                        if turnos[i][0] == turnos[j][0]:
                            bloquesTrabajos[iconductor[j],turnos[j][0]]+=1
                        else:
                            bloquesTrabajos[iconductor[j],turnos[j][0]]=1
                    #Actualizamos el diccionario de 'bloqueTabajo' 
                    else:
                        bloquesTrabajos[iconductor[j],turnos[j][0]]=1
                    #Actualizamos diccionario de guarda el rostering de la solución final
                    if dia==0:
                        M[iconductor[j]]=[turnos[j][0]]
                    else:
                        M[iconductor[j]].append(turnos[j][0])
        #En caso de que toque recalcular  
        else:
            #Desactivamos la variable graficar para no hacer ningun grafico
            graficar=False
            #Creamos lista de arcos candidatos a quitar de las asignaciones anteriores
            quitar = []
            #Lista de conductores candidatos que ya tienen la carga de trabajo completo
            candidatos = []
            #Copiamos la lista de arcos prohibidos
            proh1 = proh.copy()
            #Añadimos conductores candidatos a la lista 'candidatos'
            for i in conductores:
                if cargaTrabajo[i]==max(list(cargaTrabajo.values())):
                    candidatos.append(i)
            #Si ambos conductores de una asignación estan en un arco asignado anteriormente, añadimos ese arco a la lista de 'quitar'
            for i,j in B: 
                if iconductor[i] in candidatos or iconductor[j] in candidatos:
                    quitar.append((i,j))

            #Seleccionamos aleatoreamente el arco que vamos a quitar
            e = random.choice(quitar)
            #Restablecemos la lista de arcos de la solución final
            B = []
            #Restablecemos la lista de arcos posibles
            Arcos=[]
            #Añadimos a el arco seleccionado a la lista de prohibidos
            proh1.append(e)
            #Si no se ha superado cierto numero de iteraciones, llamamos al constructivo nuevamente para 
            #recontruir la solución teniendo en cuenta las prohibiciones
            if itera>=1:
                Constructivo(p,q,proh1,itera-1,acum+time.time()-inicio)
            break
        #Al pasar de dia la lista 'iturnos' se vuelve la lista 'jturnos'    
        #print("Tiempo Asignación",dia,time.time()-inicio_asig)  
        iturnos = jturnos.copy()
        indexRow = indexCol.copy()
     
    #Para de contabilizar el tiempo de ejecución
    D = round(time.time()+acum-inicio,2)
    #CORRECCIÓN DE SECUENCIAS PROHIBIDAS
    
    Rostering=list(M.values())

    return Respuesta,turnos,Rostering


### **- Corrector de secuencias prohibidas**

In [5]:
#Función que arregla las secuencias prohibidas en los turnos, recibe como parametros alguna solución con errores de secuencias
#retorna la solución despues de intentar corregirla, puede todavia tener errores
def corrector_secuencias(Solucion,turnos):
    #Diccionario que guarda los conductores con errores secuencias y los turnos donde se encuentra el error
    errores = {}
    #Lista de listas que guarda el cronograma de turnos de cada conductor (solamente el tipo)
    rostering_tipo_turnos = []
    #Lista de listas que guarda el cronograma de turnos de cada conductor (nombre del turno)
    rostering_turnos = [] 

    #Recorremos la solución inicial, iteramos sobre el tamaño de la solución, es lo mismo que sobre los conductores
    for i in range(len(Solucion)):
        #Creamos una lista vacía para guardar los tipos de turnos del conductor actual en la semana
        secuencia_tipo_turnos = ['-' for d in range(data['length'])]
        #Creamos una lista vacía para guardar los nombres de los turnos del conductor actual en la semana
        secuencia_turnos = ['-' for d in range(data['length'])]
        #Iteramos la cantidad de turnos asignado a ese conductor en el horizonte de planeación
        for j in range(len(Solucion[i])):
            #Obtenemos el nombre sel turno asignado a ese conductor ese dia
            nombre = Solucion[i][j]
            #Obtenemos el dia del turno
            dia = turnos[nombre][1]
            #Obtenemos el tipo del turno
            tipo = turnos[nombre][0]
            #Actualizamos la secuencia que contiene solamente los tipos de turnos de ese conductor
            secuencia_tipo_turnos[dia] = tipo
            #Actualizamos la secuencia que contiene los nombres de los turnos de ese conductor
            secuencia_turnos[dia] = nombre

        #Revisamos si la secuencia de tipos de turnos es valida, solo entra si no es valida
        if secuencia_Valida(secuencia_tipo_turnos) == False:
            #Lista que guarda los turnos que generan el error
            quitar = []
            #Revisamos cada dia de la semana
            for d in range(data['length']):
                #Obtenemos el tipo de turno asignado ese dia
                tipo = secuencia_tipo_turnos[d]
                #Lo modificamos temporalmente a descanso para ver si al quitarlo se corrige la secuencia
                secuencia_tipo_turnos[d]='-'
                #Solo se revisa en caso de que el tipo de turno que habia ese dia no fuera un descanso y en caso de corregirse la secuencia
                if tipo != '-' and secuencia_Valida(secuencia_tipo_turnos):
                    #Añadimos ese dia a los posibles turnos que podemos quitar para corregir secuencia
                    quitar.append(d)
                    #Volvemos a dejar la secuencia como estaba
                    secuencia_tipo_turnos[d]=tipo
            #Actualizamos el diccionario de errores con la información de turnos que pueden arreglar la secuencia de determinado conductor
            errores[i]=quitar

        #Actualizamos el rostering que contiene solamente los tipos de turnos
        rostering_tipo_turnos.append(secuencia_tipo_turnos)
        #Actualizamos el rostering que contiene los nombres de los turnos
        rostering_turnos.append(secuencia_turnos)

    #Variable que guarda la cantidad de errores de secuencias que hay
    Cantidad_errores = len(list(errores.keys()))
    #Variable que guarda la cantidad de errores corregidos
    Cantidad_errores_corregidos = 0
    #Solo corregimos si hay algo por corregir
    if Cantidad_errores>0:
        #Iteramos sobre los conductores que tienen secuencias prohibidas
        for conductor_corregir in list(errores.keys()):
            #Variable que indica si se pudo corregir el error
            arreglo = False
            #Iteramos sobre los turnos que se pueden quitar para corregir la secuencia
            for dia_cambio in errores[conductor_corregir]:
                #Solo seguimos si aun no se ha corregido el error
                if arreglo == False:
                    #Buscamos otro conductor
                    for c in conductores:
                        #Este conductor debe ser diferente tanto al de la secuencia prohibida como a cualquier otro que tambien tiene errores
                        if c!=conductor_corregir and c not in list(errores.keys()):
                            #OPERADOR INSERT------------------------------------------------------
                            #En caso de que el otro conductor no tenga asignado ningun turno ese dia
                            if rostering_tipo_turnos[c][dia_cambio]=='-' and arreglo == False:
                                #Creamos secuancias de apoyo para no afectar las originales
                                secuencia = rostering_tipo_turnos[c].copy()
                                secuencia[dia_cambio] = rostering_tipo_turnos[conductor_corregir][dia_cambio]
                                secuencia_turnos = rostering_turnos[c].copy()
                                secuencia_turnos[dia_cambio] = rostering_turnos[conductor_corregir][dia_cambio]
                                #Revisamos si al hacer el insert se corrigen las secuencias
                                if secuencia_Valida(secuencia,True) and arreglo == False:
                                    #En caso de que funcione acualizamos las secuencias
                                    rostering_tipo_turnos[c]=secuencia
                                    rostering_tipo_turnos[conductor_corregir][dia_cambio]='-'
                                    rostering_turnos[c]=secuencia_turnos
                                    rostering_turnos[conductor_corregir][dia_cambio]='-'
                                    arreglo = True
                                    Cantidad_errores_corregidos+=1
                                else:
                                    #OPERADOR 2 INSERT y un SWAP---------------------------------
                                    #en caso de que no funcione trataremos de insertar alguno de los turnos del segundo conductor en otro
                                    #y luego hacer un SWAP con un cuarto conductor
                                    
                                    #Buscmos un dia donde vayamos a insertar el turno del segundo conductor en el tercero, para luego hacer el swap con el cuarto
                                    for d in range(data['length']):
                                        #La unica condición es que no sea el mismo dia del turno donde se inserto el turno de corrección en el segundo conductor
                                        if d != dia_cambio and arreglo == False:
                                            #Creamos secuancias de apoyo para no afectar las originales
                                            t=secuencia[d]
                                            t2=secuencia_turnos[d]
                                            secuencia[d]='-'
                                            secuencia_turnos[d]='-'
                                            #Revisamos si al eliminar el turno del segundo conductor en ese dia se corrige las secuencias ocasionadas al insertar el turno del primer conductor
                                            if secuencia_Valida(secuencia,True) and arreglo == False:
                                                #Ahora buscamos el tercer y cuarto conductor
                                                for c1 in conductores:
                                                    for c2 in conductores:
                                                        #Deben ser todos diferentes y no ser conductores que tengan secuencias prohibidas
                                                        if c1!=c and c1!=conductor_corregir and c2!=conductor_corregir and c2!=c and c1!=c2 and c1 not in list(errores.keys()) and c2 not in list(errores.keys()) and arreglo == False:
                                                            #El tercer conductor debe estar libre ese dia porque se le insertara el turno del segundo conductor, y el cuarto si debe estar ocupado para hacer el SWAP
                                                            if rostering_tipo_turnos[c1][d] == '-' and rostering_tipo_turnos[c2][d] != '-' and arreglo == False:
                                                                #Hacemos el INSERT y SWAP en las secuencias de apoyo para no afectar las originales
                                                                secuencia1 = rostering_tipo_turnos[c1].copy()
                                                                secuencia2 = rostering_tipo_turnos[c2].copy()
                                                                secuencia_turnos1 = rostering_turnos[c1].copy()
                                                                secuencia_turnos2 = rostering_turnos[c2].copy()
                                                                secuencia1[d] = secuencia2[d]
                                                                secuencia2[d] = t
                                                                secuencia_turnos1[d] = secuencia_turnos2[d]
                                                                secuencia_turnos2[d] = t2
                                                                #Revisamos si al hacer el INSERT y SWAP se corrigen las secuencias
                                                                if secuencia_Valida(secuencia1,True) and len([elemento for elemento in secuencia1 if elemento != '-'])<=maximo_final and secuencia_Valida(secuencia2) and len([elemento for elemento in secuencia2 if elemento != '-'])<=maximo_final and arreglo == False:
                                                                    #Actualizamos las secuencias originales si funcionaron los cambios
                                                                    rostering_tipo_turnos[c1]=secuencia1
                                                                    rostering_tipo_turnos[c2]=secuencia2
                                                                    rostering_tipo_turnos[c]=secuencia
                                                                    rostering_tipo_turnos[conductor_corregir][dia_cambio]='-'
                                                                    rostering_turnos[c1]=secuencia_turnos1
                                                                    rostering_turnos[c2]=secuencia_turnos2
                                                                    rostering_turnos[c]=secuencia_turnos
                                                                    rostering_turnos[conductor_corregir][dia_cambio]='-'
                                                                    arreglo = True
                                                                    Cantidad_errores_corregidos+=1
                                                #En caso de que no funcione el operador restablecemos la secuencia del segundo conductor para seguir intentandolo en otro dia
                                                if arreglo == False:
                                                    secuencia[d]=t
                                                    secuencia_turnos[d]=t2
                                            #En caso de que no funcione el operador restablecemos la secuencia del segundo conductor para seguir intentandolo en otro dia
                                            else:
                                                secuencia[d]=t
                                                secuencia_turnos[d]=t2
    else:
        #print("Solución sin errores")
        return Solucion,turnos,rostering_tipo_turnos,True
    Solucion_corregida = [[elemento for elemento in sublista if elemento != '-'] for sublista in rostering_turnos]

    if Cantidad_errores==Cantidad_errores_corregidos and revisar_proh(rostering_tipo_turnos)==0:
        #print("Correción realizada")
        return Solucion_corregida,turnos,rostering_tipo_turnos,True
    else:
        revisar_proh(rostering_tipo_turnos)
        #print("Correción no efectiva")
        return Solucion,turnos,rostering_tipo_turnos,False


### **- Algoritmo Construcctivo pseudoaleatorio**

In [6]:
#función para la creación de la lista restricta de candidatos (RCL)
def RCL(tipo, lista, trab ,alpha):
    categorias = [[] for i in range(3)]
    personas = []
    minimo = 99999
    maximo = -1
    for i in lista:
        if trab[i]<minimo:
            minimo=trab[i]
        if trab[i]>maximo:
            maximo=trab[i]
    for i in lista:
        if trab[i]<=math.ceil(minimo+alpha*(maximo-minimo)):
            personas.append(i)
    
    for p in range(len(personas)):
        if preferencias[personas[p]][0]==tipo:
            categorias[0].append(personas[p])
        elif preferencias[personas[p]][1]==tipo:
            categorias[2].append(personas[p])
        else:
            categorias[1].append(personas[p])
    
    return categorias


#función algoritmo constructivo recibe como parametros el tamaño de la lista restricta de candidatos y si queremos que repare las soluciones obtenidas en la fase de solo construcción
def constructivo_pseudoaleatorio(alpha,reparar):
    #CONSTRUCCIÓN
    #Diccionario que guarda que conductor a sido asignado a cada turno (Los turnos son las llaves), al inicio el turno no ha sido asignado a ningun conductor entonces el valor asociado al conductor es -1
    t = {i:-1 for i in turnos}
    #Lista que guarda la carga de turnos de cada conductor, la posición en la lista equivale al conductor
    trabajados = [0 for conductor in conductores]
    #Diccionario con lista conductores disponibles por día
    ddia = {i:conductores.copy() for i in range(data['length'])}
    #Diccionario con los turnos asignados a cada conductor, se utiliza para revisar las secuencias prohibidas y restricciones de bloques
    rostering = [['-' for i in range(data['length'])] for c in conductores]
    #Variable que nos dice si se le ha asignado un turno a algun conductor, arranca en True por lo que va a ser nuestra condicion en el ciclo while
    agrego=True
    #Variable que guarda nuestra función objetivo, empieza por el momento en 0
    obj=0
    #Posición en nuestra RCL que queremos utilizar, la posición 0 son solo los conductores que les gusta el turno, la posicion 1 es a los que les es indiferente, y la posición 2 a los que no les gusta
    #Empezamos en 0 porque no interesa en primera instancia solo asignar turnos a conductores que les gusta el turno
    v=0
    #Numero de iteraciones
    itera=0
    #Lista de listas donde guardamos información de las iteraciones como los diferentes objetivos, numero de iteracion y si se esta excediendo la restricción de carga que en este caso estamos relajando
    g=[[] for i in range(3)]
    #Siempre que hayamos asignado un conductor a algun turno debemos seguir iterando
    while agrego==True:
        #Reiniciamos la variable agrego para ver si en el transcurso del ciclo asignamos a algun conductor a un turno
        agrego=False
        #Iteramos sobre cada uno de los turnos
        for tipo,dia,requerimiento in t:
            #Si el turno no ha sido asignado a ningun conductor
            if t[tipo,dia,requerimiento] ==-1:
                #Generamos lista restricta de candidatos para ser asignados en la posición v (En primera instancia solo los que les gusta, en segunda instancia, luego a los indiferentes y por ultimo a los que no les gusta)
                lista=RCL(tipo,ddia[dia],trabajados,alpha)[v]
                #Si la lista no esta vacia significa que hay candidatos
                if len(lista)>0:
                    #Seleccionamos aleatoriamente un conductor para asignar a ese turno
                    elegido = random.choice(lista)
                    #Generamos como quedaria la secuencia de ese conductor
                    secuencia = rostering[elegido].copy()
                    secuencia[dia] = tipo
                    #Revisamos que la secuencia sea valida con ese cambio
                    if secuencia_Valida(secuencia)==True:
                        #Asignamos el conductor al turno
                        t[tipo,dia,requerimiento]=elegido
                        #Actualizamos la carga del conductor en la lista de carga de trabajo
                        trabajados[elegido]+=1
                        #Actualizamos la lista de conductores disponibles para ese día (ese conductor ya no trabaja mas es dia)
                        ddia[dia].remove(elegido)
                        #Actualizamos la variable agrego porque exitosamente asignamos un turno a un conductor
                        agrego=True
                        #Actualizamos la variable obj porque se ha asignado un conductor a un turno
                        if tipo==preferencias[elegido][0]:
                            obj+=1
                        elif tipo==preferencias[elegido][1]:
                            obj+=-1/len(t)
                        #Actualizamos el numero de iteraciones
                        itera+=1
                        #Guardamos información de la iteración
                        g[0].append(obj)
                        g[1].append(itera)
                        if max(trabajados)>maximo_final:
                            g[2].append(1)
                        else:
                            g[2].append(0)
                        #Actualizamos la secuencia del conductor
                        rostering[elegido]=secuencia
        #En caso de no haber agregado ningun turno y que nuestra posicion v de la RCL sea menor a 2, actualizamos agrego para seguir iterando y incrementamos v en 1             
        if agrego==False and v<2:
            agrego=True
            v+=1

    #REPARACIÓN NO ASIGNADOS
    #Variable que me indica si entramos a la fase de reparación
    hizo=False
    #hay dos criterios indispensables para empezar con la reparación. Primero que el parámetro 𝑟𝑒𝑝𝑎𝑟𝑎𝑟 sea verdadero y segundo que la cantidad 
    #de turnos asignados sea menor al total de tunos y mayor a un porcentaje de los datos (línea 1). El porcentaje de asignaciones mínimas para
    #reparar la solución se determinó como el mismo parámetro 𝑎𝑙𝑝ℎ𝑎 del tamaño de la RCL ya que si este disminuye se espera que más soluciones 
    #deban ser reparadas y por ello este porcentaje también.
    if sum(trabajados)<len(t) and sum(trabajados)>=alpha*len(t) and reparar==True:
        #Actualizamos variable hizo
        hizo=True
        #Iteramos sobre cada uno de los turnos
        for tipo,dia,requerimiento in t:
            #Si el turno no ha sido asignado
            if t[tipo,dia,requerimiento] ==-1:
                #Inicializamo variable reparo que indica si fue posible asignar ese turno (reparar la solución). Inicia en False
                reparo=False
                #Revisamos la lista de conductores de conductores que puden ser asignados ese dia
                for elegido in ddia[dia]:
                    #Generamos como quedaria la secuencia de ese conductor en caso de asignarlo a ese turno
                    secuencia = rostering[elegido].copy()
                    secuencia[dia] = tipo
                    #Revisamos que la secuencia sea valida con ese cambio
                    if secuencia_Valida(secuencia):
                        #Asignamos el conductor al turno
                        t[tipo,dia,requerimiento]=elegido
                        #Actualizamos la carga del conductor en la lista de carga de trabajo
                        trabajados[elegido]+=1
                        #Actualizamos la lista de conductores disponibles ese dia
                        ddia[dia].remove(elegido)
                        #Actualizamos la variable agrego
                        agrego=True
                        #Actualizamos la función objetivo
                        if tipo==preferencias[elegido][0]:
                            obj+=1
                        elif tipo==preferencias[elegido][1]:
                            obj+=-1/len(t)
                        #Actualizamos la varible de reparo
                        reparo=True
                        #Aumenta la variable iteración en 1
                        itera+=1
                        #Guardamos información de la iteración
                        g[0].append(obj)
                        g[1].append(itera)
                        if max(trabajados)>maximo_final:
                            g[2].append(1)
                        else:
                            g[2].append(0)
                        #Actualizamos la secuencia del conductor
                        rostering[elegido]=secuencia
                #En caso de que no haya podido reparar sale del bucle
                if reparo==False:
                    break
    #Categorizamos solución encontrada
    r=0            
    if hizo==True:
        if sum(trabajados)==len(t):
            r=1
        else:
            r=2
    if hizo==False:
        if sum(trabajados)<len(t):
            r=3
        
    #BALANCEO EXCESO TRABAJO
    #Lista de conductores a los que les falta carga
    falta = []
    #Lista de conductores que ya tienen carga completa
    completo = []
    #Lista de conductores que debemos quitarles carga
    excede = []
    #Añadimos conductores a las listas
    for c in conductores:
        if trabajados[c]<maximo_final:
            falta.append(c)
        elif trabajados[c]==maximo_final:
            completo.append(c)
        else:
            excede.append(c)
    #Variable que me dice si el balanceo hace cambios, empezamos en True porque va a ser nuestra condición del bucle
    cambio=True
    #Iteramos mientras que se hayan hecho cambio y ya todos los turnos hayan sido asignados. Lo que significa que las soluciones deberion haber sido reparadas en su totalidad
    while cambio ==True and sum(trabajados)==len(t):
        #Actualizamos variable cambio a False para ver si durante la iteración hay algun cambio
        cambio=False
        #Iteramos sobre los turnos
        for tipo,dia,requerimiento in t:
            #Obtenemos conductor asignado al turno
            actual=t[tipo,dia,requerimiento]
            #Si el turno esta asignado a un conductor que debemos quitarle carga
            if actual in excede:
                #Obtenemos otro conductor al que se le pueda asignar mas carga
                elegido=random.choice(falta)
                #Generamos como quedaria la secuencia de ese conductor en caso de asignarlo a ese turno
                secuencia = rostering[elegido].copy()
                secuencia[dia] = tipo
                secuencia2 = rostering[actual].copy()
                secuencia2[dia] = '-'
                #Revisamos que la secuencia sea valida con ese cambio y que pueda ser asignado ese dia
                if secuencia_Valida(secuencia) and secuencia_Valida(secuencia2) and elegido in ddia[dia]:
                    #Asignamos el turno al conductor elegido
                    t[tipo,dia,requerimiento]=elegido
                    #Actualizamos las cargas de ambos conductores
                    trabajados[elegido]+=1
                    trabajados[actual]+=-1
                    #Actualizamos las listas de conductores disponibles cada dia
                    ddia[dia].remove(elegido)
                    ddia[dia].append(actual)
                    #Actualizamos la variable agrego
                    agrego=True
                    #Actualizamos función objetivo
                    if tipo==preferencias[actual][0]:
                        if tipo!=preferencias[elegido][0]:
                            if tipo==preferencias[elegido][1]:
                                obj+=-(1+1/len(t))
                            else:
                                obj+=-1
                    elif tipo==preferencias[actual][1]:
                        if tipo!=preferencias[elegido][1]:
                            if tipo==preferencias[elegido][0]:
                                obj+=(1+1/len(t))
                            else:
                                obj+=1/len(t)
                    else:
                        if tipo==preferencias[elegido][0]:
                            obj+=1
                        elif tipo==preferencias[elegido][1]:
                            obj+=-1/len(t)
                    #Aumenta la variable iteración en 1  
                    itera+=1
                    #Guardamos información de la iteración
                    g[0].append(obj)
                    g[1].append(itera)
                    if max(trabajados)>maximo_final:
                        g[2].append(1)
                    else:
                        g[2].append(0)
                    #Actualizamos las listas que categorizan la carga de los conductores 
                    if trabajados[elegido]==maximo_final:
                        completo.append(elegido)
                        falta.remove(elegido)
                    if trabajados[actual]==maximo_final:
                        completo.append(actual)
                        excede.remove(actual)
                    cambio=True
                    #Actualizamos la secuencia del conductor
                    rostering[elegido]=secuencia
                    rostering[actual]=secuencia2
    
    #BALANCEO FALTA DE TRABAJO
    #Lista de conductores que no cumplen con el minimo de dias trabajados
    falta_incumple = [c for c in falta if trabajados[c]<maximo_inicial]
    revisiones = 0
    max_revisiones = len(falta_incumple)+1
    while len(falta_incumple)>0:
        revisiones+=1
        #Iteramos sobre los turnos
        for tipo,dia,requerimiento in t:
            #Obtenemos conductor asignado al turno
            actual=t[tipo,dia,requerimiento]
            #Si el turno esta asignado a un conductor que debemos quitarle carga
            if actual in excede or (actual in completo and trabajados[actual]>=maximo_inicial+1):
                #Obtenemos otro conductor al que debamos asignarle mas carga
                elegido=random.choice(falta_incumple)
                #Generamos como quedaria la secuencia de ese conductor en caso de asignarlo a ese turno
                secuencia = rostering[elegido].copy()
                secuencia[dia] = tipo
                secuencia2 = rostering[actual].copy()
                secuencia2[dia] = '-'
                #Revisamos que la secuencia sea valida con ese cambio y que pueda ser asignado ese dia
                if secuencia_Valida(secuencia) and secuencia_Valida(secuencia2) and elegido in ddia[dia]:
                    #Asignamos el turno al conductor elegido
                    t[tipo,dia,requerimiento]=elegido
                    #Actualizamos las cargas de ambos conductores
                    trabajados[elegido]+=1
                    trabajados[actual]+=-1
                    #Actualizamos las listas de conductores disponibles cada dia
                    ddia[dia].remove(elegido)
                    ddia[dia].append(actual)
                    #Actualizamos la variable agrego
                    agrego=True
                    #Actualizamos función objetivo
                    if tipo==preferencias[actual][0]:
                        if tipo!=preferencias[elegido][0]:
                            if tipo==preferencias[elegido][1]:
                                obj+=-(1+1/len(t))
                            else:
                                obj+=-1
                    elif tipo==preferencias[actual][1]:
                        if tipo!=preferencias[elegido][1]:
                            if tipo==preferencias[elegido][0]:
                                obj+=(1+1/len(t))
                            else:
                                obj+=1/len(t)
                    else:
                        if tipo==preferencias[elegido][0]:
                            obj+=1
                        elif tipo==preferencias[elegido][1]:
                            obj+=-1/len(t)
                    #Aumenta la variable iteración en 1  
                    itera+=1
                    #Guardamos información de la iteración
                    g[0].append(obj)
                    g[1].append(itera)
                    if max(trabajados)>maximo_final:
                        g[2].append(1)
                    else:
                        g[2].append(0)
                    #Actualizamos las listas que categorizan la carga de los conductores 
                    if trabajados[elegido]==maximo_inicial:
                        falta_incumple.remove(elegido)
                        if len(falta_incumple)==0:
                            break
                    #Actualizamos la secuencia del conductor
                    rostering[elegido]=secuencia
                    rostering[actual]=secuencia2
        if revisiones==max_revisiones:
            #print(f'{len(falta_incumple)} conductores no tienen minimo de dias de trabajo')
            break
    return t,trabajados,sum(trabajados),obj,g,r,rostering

### **- Busqueda Local**

In [7]:
##  BUSQUEDA LOCAL ##############################################################################################################################
#Funcion de busqueda local recibe como parametro el diccionario que me dice a que conductor ha sido asignado cada turno. la función objetivo de la 
#solución actual, la lista de carga de trabajo actual y la información de las iteraciones del constructivo y el rostering para revisar secuencias
def busqueda_local(turns,obj,trab,g,rostering):
    #Calculo de satisfechos y insatisfechos para saber como va cambiando el puntaje
    satisfechos = 0
    insatisfechos = 0
    total = 0
    for i in range(len(rostering)):
        for t in rostering[i]:
            if t == preferencias[i][0]:
                total += 1
                satisfechos+=1
            elif t == preferencias[i][1]:
                total += 1
                insatisfechos+=1
            else:
                total += 1

    #Diccionario que para cada dia contiene una lista de conductores disponibles
    ddia = {i:conductores.copy() for i in range(data['length'])}
    #Actualizamos listas de conductores disponibles leyendo la solución dada por parametro
    for t,d,r in turns:
        if turns[t,d,r] in ddia[d]:
            ddia[d].remove(turns[t,d,r])
    #Inicializamos variable mejoro en True ya que vamos es nuestra condición para seguir iterando
    mejoro = True
    #Iteramos hasta que no mejoremos la solución
    while mejoro==True:
        #Actualizamos variable mejoro para ver si en algun momento de la busqueda mejoramos la solución
        mejoro = False
        #Inicializamos el numero de la iteración teniendo en cuenta el numero en que vamos en nuestra lista g[1]
        itera=max(g[1])
        #Iteramos sobre los posibles turnos
        for t,d,r in turns:
            #Obtenemos el conductor asignado a ese turno
            elec=turns[t,d,r]
            #Obtenemos el turno que le gusta a ese conductor
            si=preferencias[elec][0]
            #Obtenemos el turno que le disgusta a ese conductor
            no=preferencias[elec][1]

            ####################  INSERT  ##########################
            #Vamos a tratar de insertar otro conductor a ese turno, por lo que iteramos sobre la lista de conductores
            for c in conductores.copy():
                #Siempre y cuando el conductor sea diferente al que ya esta asignado
                if c!=elec:
                    #Generamos como quedaria la secuencia de ese conductor en caso de asignarlo a ese turno
                    secuencia = rostering[elec].copy()
                    secuencia[d] = '-'
                    secuencia2 = rostering[c].copy()
                    secuencia2[d] = t
                    #Revisamos que al hacer la inserción se sigan cumpliendo las restricciones
                    if trab[c]<=maximo_final-1 and secuencia_Valida(secuencia) and secuencia_Valida(secuencia2) and c in ddia[d] and trab[elec]>=maximo_inicial+1:
                        #Revisamos como aporta la nueva solución en comparación a la nueva
                        sic=preferencias[c][0]
                        noc=preferencias[c][1]
                        satisfechos1=satisfechos
                        insatisfechos1=insatisfechos
                        if sic==t:
                            satisfechos1+=1
                        elif noc==t:
                            insatisfechos1+=1
                        if si==t:
                            satisfechos1-=1
                        elif no==t:
                            insatisfechos1-=1
                        obj1 = satisfechos1-insatisfechos1/total
                        #En caso de que mejore hacemos la inserción de ese conductor al turno
                        if obj1>obj:
                            #Actualizamos la función objetivo
                            #obj+=(mejora1-mejora)
                            obj=obj1
                            satisfechos=satisfechos1
                            insatisfechos=insatisfechos1
                            #Actualizamos la variable mejor
                            mejoro = True
                            #Asignamos ese conductor al turno
                            turns[t,d,r]=c
                            #Actualizamos listas de disponibilidad de conductores por dia
                            ddia[d].remove(c)
                            ddia[d].append(elec)
                            #Actualizamos la lista de carga de trabajo
                            trab[elec]+=-1
                            trab[c]+=1
                            #Actualizamos la secuencia del conductor
                            rostering[elec]=secuencia
                            rostering[c]=secuencia2
                            #print("INSERT",c,"-->",obj)
                            elec=c
                            #Actualizamos preferencias del conductor
                            si=preferencias[elec][0]
                            no=preferencias[elec][1]
                            #Aumenta la variable iteración en 1  
                            itera+=1
                            #Guardamos información de la iteración
                            g[0].append(obj)
                            g[1].append(itera)
                            g[2].append(0)
                            break

            ####################  SWAP  ##########################
            #Ahora vamos a tratar de intercambiar conductores entre turnos
            #Para ello iteramos otra vez en los turnos
            for t1,d1,r1 in turns:
                #Solo es valido el SWAP si el tipo de turno es diferente, o en caso de que el tipo sea igual que el dia sea tambien igual
                if  (t!=t1 or(t==t1 and d==d1)):
                    #Obtenemos el conductor asignado al otro turno
                    elec1=turns[t1,d1,r1]
                    #Revisamos que ambos conductores esten disponibles el dia del SWAP
                    if elec in ddia[d1] and elec1 in ddia[d]:
                        #Generamos como quedaria la secuencia de ese conductor en caso de asignarlo a ese turno
                        secuencia = rostering[elec].copy()
                        secuencia[d1] = t1
                        secuencia[d] = '-'
                        secuencia2 = rostering[elec1].copy()
                        secuencia2[d] = t
                        secuencia2[d1] = '-'
                        #Revisamos que ambos conductores no tengan secuencias prohibidas al hacer el SWAP
                        if secuencia_Valida(secuencia) and secuencia_Valida(secuencia2):
                            #Revisamos como aporta la nueva solución en comparación a la nueva
                            si1=preferencias[elec1][0]
                            no1=preferencias[elec1][1]
                            satisfechos1=satisfechos
                            insatisfechos1=insatisfechos
                            if si==t1:
                                satisfechos1+=1
                            elif no==t1:
                                insatisfechos1+=1
                            if si1==t:
                                satisfechos1+=1
                            elif no1==t:
                                insatisfechos1+=1

                            if si==t:
                                satisfechos1-=1
                            elif no==t:
                                insatisfechos1-=1
                            if si1==t1:
                                satisfechos1-=1
                            elif no1==t1:
                                insatisfechos1-=1

                            obj1=satisfechos1-insatisfechos1/total
                            #En caso de que mejore hacemos la inserción de ese conductor al turno
                            if obj1>obj:
                                #Actualizamos función objetivo
                                obj=obj1 
                                satisfechos=satisfechos1
                                insatisfechos=insatisfechos1
                                #Actualizamos la variable mejoro
                                mejoro = True
                                #Asignamos los turnos despues del SWAP
                                turns[t,d,r]=elec1
                                turns[t1,d1,r1]=elec
                                #Actualizamos listas de conductores disponibles por dia
                                ddia[d].remove(elec1)
                                ddia[d].append(elec)
                                ddia[d1].remove(elec)
                                ddia[d1].append(elec1)
                                #Actualizamos las secuencias de conductores
                                rostering[elec]=secuencia
                                rostering[elec1]=secuencia2
                                #print("SWAP",elec,elec1,"-->",obj)
                                #Aumenta la variable iteración en 1  
                                itera+=1
                                #Guardamos información de la iteracióng[0].append(obj)
                                g[0].append(obj)
                                g[1].append(itera)
                                g[2].append(0)
                                break
    return turns, obj,trab,g,rostering

### **- GRASP**

In [8]:
##  GRASP  ###############################################################################################################
#Función algoritmo GRASP, recibe como parametro el numero de iteraciones, el tamaño de la RCL y el umbral minimo de iteraciones para reparar soluciones
def GRASP(itera,alpha,umbral):
    #Variable que guarda mejor función objetivo encontrada
    max_v=0
    #Variable que guarda mejor disposición de turnos encontrada
    max_t=[]
    #Contadores de soluciones exitosas, no exitosas, reparadas y reparaciones fallidas
    r0=0
    r1=0
    r2=0
    r3=0
    max_tiempo=0
    #Empezamos sin reparar ninguna solución del constructiva hasta llegar al umbral
    reparar=False
    #Empezamos a contabilizar el tiempo de ejecución GRASP
    inicio=time.time()
    #Iteramos el número de veces que queremos
    for i in range(itera):
        #Si llegamos al umbral de iteraciones y no hemos encontrado ninguna solución factible empezamos a reparar
        if i>=itera*umbral and max_t==[]:
            reparar=True
        #Empezamos a contabilizar el tiempo de ejecución constructivo
        inicio_constructivo=time.time()
        #Corremos algoritmo constructivo
        t=constructivo_pseudoaleatorio(alpha,reparar)
        #Finalizamos tiempo de ejecución del constructivo
        tiempo_constructivo=inicio_constructivo-time.time()
        #Dependiendo de lo obtenido clasificamos la solución del constructivo
        if t[5]==0:
            r0+=1
        elif t[5]==1:
            r1+=1
        elif t[5]==2:
            r2+=1
        else:
            r3+=1
        #Si hemos asignado exitosamente todos los turnos podemos seguir a la busqueda local
        if sum(t[1])==len(t[0]):
            #Creamos diccionario que guarda solucion del constructivo
            asignacion = {}
            for turno in t[0]:
                asignacion[turno]=(t[0][turno],turno)
            #Obtenemos funcion objetivo
            v=t[3]
            #Finalizamos tiempo de ejecución de la busqueda local
            inicio_bl=time.time()
            #Corremos algoritmo busqueda local
            t1=busqueda_local(t[0],v,t[1],t[4],t[6])
            #Finalizamos tiempo de ejecución de la busqueda local
            tiempo_bl=inicio_bl-time.time()
            #Si la solución encontrada es mejor que la mejor encontrada hasta el momento la guarda como la mejor
            if t1[1]>max_v and max(t1[2])<=maximo_final:
                max_v=t1[1]
                max_t=[t,t1,v]
                max_tiempo=(tiempo_constructivo,tiempo_bl)
    #Guada el tiempo de ejecución del GRASP
    D = round(time.time()-inicio,4)

    return max_v,max_t,r0,r1,r2,r3,D,max_tiempo

### **- Split**

In [9]:
def generador_Rostering(rutas,nturnos,turnos,data):
    Rostering = []
    for conductor in rutas:
        rostering_conductor = ['-' for i in range(data['length'])]
        for turno in conductor:
            rostering_conductor[turnos[nturnos[turno]][1]]=turnos[nturnos[turno]][0]
        Rostering.append(rostering_conductor)
    return Rostering

def calcular_distancias_acumuladas(distancias,nturnos):
    """Calcula las distancias acumuladas entre clientes."""
    D = [0] * (len(nturnos) )
    for i in range(1, len(nturnos) ):
        D[i] = D[i-1] + distancias[(nturnos[i-1],nturnos[i])]
    return D

def calcular_demandas_acumuladas(demandas):
    """Calcula las demandas acumuladas de los clientes."""
    Q = [0] * (len(demandas) ) 
    for i in range(1, len(demandas) ):
        Q[i] = Q[i-1] + demandas[i]
    return Q

def calcular_bloques_acumulados(n,turnos,nturnos,tipo_turnos):
    """Calcula el número de bloques de trabajo acumulados de cada tipo de turno"""
    B = {i:[0]*n for i in tipo_turnos}
    for i in tipo_turnos:
        for j in range(1, n ):
            if turnos[nturnos[j]][0] == i and turnos[nturnos[j-1]][0] == i and turnos[nturnos[j]][1]-1==turnos[nturnos[j-1]][1]:
                B[i][j] = B[i][j-1]+1
            else:
                B[i][j] = 1
    return B

def calcular_costo_arco(p, D, Q, i, j,capacidad,dist_depot):
    """Calcula el costo de un arco (i, j) dado que D es la distancia acumulada."""
    # Evitar desbordamiento asegurándonos que i+1 y j estén dentro del rango
    if Q[j]-Q[i]<=capacidad:
        c = dist_depot[i+1] + D[j] - D[i+1] + dist_depot[j]
        return c
    return float('inf')  # Retorna infinito si los índices no son válidos

def domina(p, D, dist_depot, Q, i, j):
    """Determina si el nodo i domina al nodo j."""
    # Evitar desbordamiento asegurándonos que i+1 y j+1 estén dentro del rango
    if i+1<len(D) and j+1<len(D):
        if i<=j:
            return (p[i] + dist_depot[i+1] - D[i+1]) <= (p[j] + dist_depot[j+1] - D[j+1]) and Q[i]==Q[j]
        elif i>j:
            return (p[i] + dist_depot[i+1] - D[i+1]) <= (p[j] + dist_depot[j+1] - D[j+1])
    return False

def Particiones_Factibles_Crew_Scheduling_Split(distancias, demandas, capacidad, dist_depot,nturnos,tipo_turnos,data,turnos):
    n = len(demandas)  # Número de nodos
    p = [float('inf')] * n  # Costos mínimos hasta cada nodo
    p[0] = 0  # Costo del depósito al primer nodo
    pred = [-1] * n  # Predecesores para rastrear las rutas
    #Precalculos
    D = calcular_distancias_acumuladas(distancias,nturnos)
    Q = calcular_demandas_acumuladas(demandas)
    B = calcular_bloques_acumulados(n,turnos,nturnos,tipo_turnos)
    # Inicializamos la cola doble (deque)
    deque_cola = deque([0])
    # Iteramos sobre cada cliente
    for t in range(1, n):
        # Obtenemos el mejor predecesor actual
        mejor_pred = deque_cola[0] 
        # Calculamos el costo del arco desde el mejor predecesor
        p[t] = p[mejor_pred] + calcular_costo_arco(p, D, Q, mejor_pred, t, capacidad, dist_depot)
        pred[t] = mejor_pred
        # Si no es el último nodo, verificamos dominancia
        if t < n - 1:
            if not domina(p, D, dist_depot, Q, deque_cola[-1],t):
                # Si el nuevo nodo no domina, verificamos si hay que eliminar predecesores
                while deque_cola and domina(p, D, dist_depot, Q, t, deque_cola[-1]):
                    deque_cola.pop()
                # Añadimos el nodo t a la cola
                deque_cola.append(t)    
                # Verificamos que no haya ninguna secuencia prohibida
                agregar = True
                dia = turnos[nturnos[t+1]][1]
                if turnos[nturnos[t]][1] < dia - 1:
                    if ('-',turnos[nturnos[t+1]][0]) in data['forbidden_sequences']:
                        agregar = False
                    if turnos[nturnos[t]][1] == dia - 2:
                        if (turnos[nturnos[t]][0],'-',turnos[nturnos[t+1]][0]) in data['forbidden_sequences']:
                            agregar = False
                    elif turnos[nturnos[t]][1] < dia - 2:
                        if ('-','-',turnos[nturnos[t+1]][0]) in data['forbidden_sequences']:
                            agregar = False
                elif turnos[nturnos[t]][1] == dia - 1:
                    if (turnos[nturnos[t]][0],turnos[nturnos[t+1]][0]) in data['forbidden_sequences']:
                        agregar = False
                    if turnos[nturnos[t-1]][1] == dia - 2:
                        if (turnos[nturnos[t-1]][0],turnos[nturnos[t]][0],turnos[nturnos[t+1]][0]) in data['forbidden_sequences']:
                            agregar = False
                    elif turnos[nturnos[t-1]][1] < dia - 2:
                        if ('-',turnos[nturnos[t]][0],turnos[nturnos[t+1]][0]) in data['forbidden_sequences']:
                            agregar = False
                else:
                    agregar = False
                #En caso de que se incumpla alguna secuencia
                if agregar==False:
                    while deque_cola[0] != t:
                        deque_cola.popleft()
            # Verificamos si el nodo actual excede la capacidad de turnos de trabajo
            while deque_cola and Q[t+1] - Q[deque_cola[0]] > capacidad:
                deque_cola.popleft()
            # Verificamos si el nodo actual excede la capacidad de bloques de trabajo para ese tipo de turno
            for i in range(len(tipo_turnos)):
                tipo = tipo_turnos[i]
                while deque_cola and B[tipo][t+1] - B[tipo][deque_cola[0]] > data['shift_data'][i]['employee_requirements'][1]:
                    deque_cola.popleft()
    # Reconstruimos las rutas
    rutas = reconstruir_rutas(pred,n)
    # Generamos rostering de turnos
    Rostering = generador_Rostering(rutas,nturnos,turnos,data)
    #Calculamos puntaje para medir satisfacción y insatisfacción
    satisfechos = 0
    insatisfechos = 0
    for i in range(len(Rostering)):
        for j in range(len(Rostering[i])):
            if i<len(conductores):
                if preferencias[i][0]==Rostering[i][j]:
                    satisfechos+=1
                elif preferencias[i][1]==Rostering[i][j]:
                    insatisfechos+=1
            else:
                insatisfechos+=1
    puntaje = satisfechos - insatisfechos/(p[-1]-len(rutas))
    return rutas,puntaje,Rostering  # Retorna las rutas, el costo total mínimo y el rostering

def reconstruir_rutas(pred, n):
    rutas = []
    t = n - 1  # Iniciamos desde el último nodo
    #Mientras que no hayamos llegado al nodo deposito
    while t > 0:
        #Obtenemos turnos en la ruta a traves de la precedencia
        rutas.append(list(range(pred[t]+1,t+1)))
        #Actualizamos precedencia siguiente ruta
        t = pred[t]

    rutas = rutas[::-1] 
    return rutas

def calcular_puntajes_swap(i,j,turnos,nturnos_busqueda,n):
    puntaje_actual = 0
    puntaje_swap = 0
    if turnos[nturnos_busqueda[i]][1]<turnos[nturnos_busqueda[i+1]][1]:
        puntaje_actual += 1
    if i>1:
        if turnos[nturnos_busqueda[i-1]][1]<turnos[nturnos_busqueda[i]][1]:
            puntaje_actual += 1
    if j<n-1:
        if turnos[nturnos_busqueda[j]][1]<turnos[nturnos_busqueda[j+1]][1]:
            puntaje_actual += 1
    if turnos[nturnos_busqueda[j-1]][1]<turnos[nturnos_busqueda[j]][1]:
        puntaje_actual += 1
    
    
    if turnos[nturnos_busqueda[j]][1]<turnos[nturnos_busqueda[i+1]][1]:
        puntaje_swap += 1
    if i>1:
        if turnos[nturnos_busqueda[i-1]][1]<turnos[nturnos_busqueda[j]][1]:
            puntaje_swap += 1
    if j<n-1:
        if turnos[nturnos_busqueda[i]][1]<turnos[nturnos_busqueda[j+1]][1]:
            puntaje_swap += 1
    if turnos[nturnos_busqueda[j-1]][1]<turnos[nturnos_busqueda[i]][1]:
        puntaje_swap += 1
    
    return puntaje_actual,puntaje_swap


def Crew_Scheduling_Split(solucion,turnos,Tiempo_busqueda):
    #Diccionario de turnos Split
    turnosSplit = turnos.copy()
    #Creamos una lista que guarda los turnos en la secuencia de la solucion actual
    turnosOrder = []
    #Añadimos los elementos al diccionario 'turnos' y a la lista 'nturnos'
    for i in range(len(solucion)):
        for turno in solucion[i]:
            turnosOrder.append(turno)
    #Añadismos deposito como un nodo fictio llamado 'Source'
    turnosOrder.insert(0,"Source")
    turnosSplit["Source"] = ('NO',-1,0)
    #Cantidad total de turnos
    n = len(turnosOrder)
    #Parametros necesarios para encontar particion factible teniendo en cuenta el orden de visita de los turnos
    capacidad = maximo_final
    dist_depot = [1 for i in range(len(turnosOrder))]
    demandas = [1 for i in range(len(turnosOrder))]
    distancias = {}
    for i in range(len(turnosOrder)):
        for j in range(len(turnosOrder)):
            if turnosSplit[turnosOrder[i]][1] > turnosSplit[turnosOrder[j]][1]:
                distancias[turnosOrder[i],turnosOrder[j]] = 1e10
            else:
                distancias[turnosOrder[i],turnosOrder[j]] = 1
    
    #Empieza a contabilizar el tiempo de ejecución
    inicio=time.time()

    rutas,puntaje,rostering = Particiones_Factibles_Crew_Scheduling_Split(distancias,demandas,capacidad,dist_depot,turnosOrder,tipo_turnos,data,turnosSplit)
    puntaje_inicio = puntaje
    max_k = 3
    mejoro = True
    #Comienza la busqueda Local
    while mejoro:
        mejoro = False
        inicio_insert = 1
        fin_insert = n
        if len(rutas)>len(conductores):
            inicio_insert = rutas[len(conductores)][0]
            fin_insert = inicio_insert
        for i in range(inicio_insert,n):
            for j in range(1,fin_insert):
                if i != j:
                    if len(rutas)==len(conductores):
                        if i<j and abs(i-j)>1:
                            puntaje_actual,puntaje_swap = calcular_puntajes_swap(i,j,turnosSplit,turnosOrder,n)
                            if puntaje_swap >= puntaje_actual:
                                #Hacer el SWAP
                                turno_i = turnosOrder[i]
                                turno_j = turnosOrder[j]
                                turnosOrder[i],turnosOrder[j] = turno_j,turno_i
                                rutasCambio,puntajeCambio,rosteringCambio = Particiones_Factibles_Crew_Scheduling_Split(distancias,demandas,capacidad,dist_depot,turnosOrder,tipo_turnos,data,turnosSplit)
                                #En caso de mejorar la solución actualizamos la mejor solución encontrada
                                if len(rutasCambio)<len(rutas) or (len(rutasCambio)==len(rutas) and puntajeCambio>puntaje):
                                    #print(f"SWAP {i}-{j} -> {puntajeCambio}")
                                    #print("\t",len(rutasCambio),turnosOrder)
                                    #print("\t",rutasCambio)
                                    puntaje = puntajeCambio
                                    rutas = rutasCambio
                                    rostering = rosteringCambio
                                    mejoro = True
                                    break
                                #De lo contrario devolvemos los cambios realizados
                                else:
                                    turnosOrder[i],turnosOrder[j]= turno_i,turno_j
                    else:
                        #Hacer el k-INSERT
                        for k in range(1,max_k+1):
                            if (i<j and i+k<j) or (j<i and i+k<n):
                                #Hacer el k-INSERT
                                turnos_i = turnosOrder[i:i+k]
                                for t in range(len(turnos_i)):
                                    turnosOrder.insert(j+t,turnos_i[t])

                                for t in range(len(turnos_i)):  
                                    if i<j:
                                        turnosOrder.pop(i)
                                    else:
                                        turnosOrder.pop(i+k)
                                rutasCambio,puntajeCambio,rosteringCambio = Particiones_Factibles_Crew_Scheduling_Split(distancias,demandas,capacidad,dist_depot,turnosOrder,tipo_turnos,data,turnosSplit)
                                #En caso de mejorar la solución actualizamos la mejor solución encontrada
                                if len(rutasCambio)<len(rutas) or (len(rutasCambio)==len(rutas) and puntajeCambio>puntaje):
                                    #print(f"INSERT {i}-{j} k={k} -> {puntajeCambio}")
                                    #print("\t",len(rutasCambio),turnosOrder)
                                    #print("\t",rutasCambio)
                                    puntaje = puntajeCambio
                                    rutas = rutasCambio
                                    rostering = rosteringCambio
                                    mejoro = True
                                    break
                                #De lo contrario devolvemos el INSERT
                                else:
                                    for t in range(len(turnos_i)):
                                        if i<j:
                                            turnosOrder.pop(j-k)
                                        else:
                                            turnosOrder.pop(j)

                                    for t in range(len(turnos_i)):
                                        turnosOrder.insert(i+t,turnos_i[t])   
                                        
                            if time.time()-inicio > Tiempo_busqueda:
                                break   
                    if time.time()-inicio > Tiempo_busqueda:
                        break
            if time.time()-inicio > Tiempo_busqueda:
                break
        if time.time()-inicio > Tiempo_busqueda:
            break
    
    return puntaje_inicio,puntaje, rostering

### **- Heuristica Crew Scheduling**

In [10]:
#Función algoritmo que contiene todos el trabajo de crew_schedulign
def crew_scheduling_heuristic():
    solucion=GRASP(10,0.84,0.5)
    #Mejor puntaje obtenido
    max_v=solucion[0]
    #Información mejor solución
    max_t=solucion[1]
    #Numero soluciones exitosas
    r0=solucion[2]
    #Numero soluciones no exitosas
    r1=solucion[3]
    #Numero soluciones reparadas
    r2=solucion[4]
    #Numero soluciones reparación fallida
    r3=solucion[5]
    #Tiempo ejecución GRASP
    D=solucion[6]
    inicio = time.time()
    #Ejecutamos algoritmo asignacion secuencial
    solucion1,turnos1,rostering1 = Constructivo(2,1,[],50,0)
    #Ejecutamos correcion de secuencias prohibidas con la solucion de la asignacion secuencial
    solucion2,turnos2,rostering2,factible = corrector_secuencias(solucion1,turnos1)
    tiempo_asignacion_secuencial = round(time.time()-inicio,4)
    inicio = time.time()
    #Ejecutamos algoritmo SPLIT a partir de la solucion entrega despues de tratar de corregir las secuencias prohibidas
    puntaje_inicio, puntaje_Split, rostering_Split = Crew_Scheduling_Split(solucion2,turnos2,abs(tiempo_asignacion_secuencial-D))
    tiempo_split = round(time.time()-inicio,4)
    #Pasamos información a la función graficar para que muestre la mejor solución encontrada
    graficar(max_v,max_t,r0,r1,r2,r3,D,solucion2,turnos2,rostering2,factible,tiempo_asignacion_secuencial,puntaje_inicio,puntaje_Split, rostering_Split,tiempo_split)


### **- Modelo Exacto Crew Scheduling**

In [11]:

def modelo_exacto():
    # Empezamos a cronometrar cuanto se demora el modelo en correr
    inicio = time.time()

    # Creamos el modelo de PuLP y especificamos Gurobi como solucionador
    model = pl.LpProblem("Crew_Scheduling", pl.LpMaximize)
    #solver_list = pl.listSolvers()
    #print(solver_list)
    solver = pl.GUROBI_CMD(msg=0)

    # Creamos listas para guardar tuplas de las posibles asignaciones y las clasificamos en satisfechos, indiferentes, insatisfechos y una lista que guarda todas las tuplas llamada turnos
    satisfechos = []
    indiferentes = []
    insatisfechos = []
    turnos = []

    # Variable que guarda el número de turnos que se deben asignar
    tamaño = 0
    prohibidas = data['forbidden_sequences'].copy()
    tipos = tipo_turnos.copy()
    tipos.append('-')
    num_tipos = len(tipos)

    # Recorremos la lista de tipos de turnos
    for tipo_index in range(num_tipos):
        tipo = tipos[tipo_index]
        for dia in range(data['length']):
            if tipo != '-':
                tamaño += int(data['temporal_requirements'][tipo_index][dia])
            for conductor in conductores:
                turnos.append((conductor, tipo, dia))
                if tipo != '-':
                    if tipo == preferencias[conductor][0]:
                        satisfechos.append((conductor, tipo, dia))
                    elif tipo == preferencias[conductor][1]:
                        insatisfechos.append((conductor, tipo, dia))
                    else:
                        indiferentes.append((conductor, tipo, dia))

    # VARIABLES DE DECISIÓN
    x = pl.LpVariable.dicts("x", ((conductor, tipo, dia) for conductor, tipo, dia in turnos), cat=pl.LpBinary)

    # FUNCIÓN OBJETIVO
    model += pl.lpSum(x[conductor, tipo, dia] for conductor, tipo, dia in satisfechos) - pl.lpSum(x[conductor, tipo, dia] for conductor, tipo, dia in insatisfechos) / tamaño

    # RESTRICCIONES
    for tipo_index in range(num_tipos):
        tipo = tipos[tipo_index]
        if tipo != '-':
            for dia in range(data['length']):
                model += pl.lpSum(x[conductor, tipo, dia] for conductor in conductores) == int(data['temporal_requirements'][tipo_index][dia])
            max_bloque = data['shift_data'][tipo_index]['employee_requirements'][1]
            secuencia = tuple([tipo for _ in range(max_bloque)])
            prohibidas.append(secuencia)

    for conductor in conductores:
        model += pl.lpSum(x[conductor, tipo, dia] for tipo in tipo_turnos for dia in range(data['length'])) <= maximo_final
        model += pl.lpSum(x[conductor, tipo, dia] for tipo in tipo_turnos for dia in range(data['length'])) >= int(data['min_work_blocks'])
        for dia in range(data['length']):
            model += pl.lpSum(x[conductor, tipo, dia] for tipo in tipos) == 1
            for tipo_index in range(num_tipos):
                tipo = tipos[tipo_index]
                if tipo != '-':
                    max_bloque = data['shift_data'][tipo_index]['employee_requirements'][1]
                    model += pl.lpSum(x[conductor, tipo, d] for d in range(dia, min(dia + max_bloque + 1, data['length']))) <= max_bloque
                    model += pl.lpSum(x[conductor, tipo, d] for d in range(dia, min(dia + max_bloque, data['length']))) <= max_bloque
            for proh in data['forbidden_sequences']:
                if dia + len(proh) - 1 < data['length']:
                    model += pl.lpSum(x[conductor, proh[i], dia + i] for i in range(len(proh))) <= len(proh) - 1

    # Resolución del modelo
    model.solve()
    #model.solve(solver)


    # Indicadores de la solución
    if pl.LpStatus[model.status] == 'Optimal':
        dt = []
        M = []
        for conductor in conductores:
            ruta = []
            carga = 0
            for dia in range(data['length']):
                encontro = False
                for tipo in tipos:
                    if pl.value(x[conductor, tipo, dia]) == 1:
                        ruta.append(tipo)
                        encontro = True
                        if tipo != '-':
                            carga += 1
            dt.append(carga)
            M.append(ruta)

        tabla = pd.DataFrame(M, columns=["L", "M", "X", "J", "V", "S", "D"])
        tabla_con_indices = tabla.set_index(pd.Index(["EMPLEADO " + str(i + 1) for i in range(len(conductores))]))
        estilos = [dict(selector="th", props=[("border", "1px solid black")]), dict(selector="td", props=[("border", "1px solid black")])]
        tabla_estilizada = tabla_con_indices.style.set_table_styles(estilos)

        tiempo_final = time.time() - inicio
        print("----------------------------------------------")
        print(f'Tiempo(s) = {tiempo_final}')
        print(f'Numero de turnos = {tamaño}')
        print(f'Puntaje = {round(pl.value(model.objective), 4)}')
        print(f'Satisfechos = {round((sum(pl.value(x[conductor, tipo, dia]) for conductor, tipo, dia in satisfechos) / tamaño) * 100, 2)}%')
        print(f'Indiferentes = {round((sum(pl.value(x[conductor, tipo, dia]) for conductor, tipo, dia in indiferentes) / tamaño) * 100, 2)}%')
        print(f'Insatisfechos = {round((sum(pl.value(x[conductor, tipo, dia]) for conductor, tipo, dia in insatisfechos) / tamaño) * 100, 2)}%')
        print("----------------------------------------------")
        print(revisar_proh(M),"Errores de secuencias prohibidas")
        print(sum(pl.value(x[conductor, tipo, dia]) for conductor, tipo, dia in satisfechos)+sum(pl.value(x[conductor, tipo, dia]) for conductor, tipo, dia in indiferentes)+sum(pl.value(x[conductor, tipo, dia]) for conductor, tipo, dia in insatisfechos),"turnos asignados")
        print("# Dias de trabajo (min y max):",min(dt),"-",max(dt),"(",data['min_work_blocks'],"-",data['max_work_blocks'],")")
        print("# Dias de descanso (min y max):",data['length']-max(dt),"-",data['length']-min(dt),"(",data['min_days_off'],"-",data['max_days_off'],")")
        print("----------------------------------------------")
        display(tabla_estilizada)

    else:
        print("INFACTIBLE")
        print(pl.LpStatus[model.status])

## **SOLUCIONAR :**

### **- Heuristica**

In [12]:
crew_scheduling_heuristic()

,Exitosas,No exitosas,Reparadas,Reparación fallida
#,0,5,0,5


Tiempo de ejecución total -> 2.9263 s


,Tiempo (s),Puntaje Constructivo Asignación,Satisfechos,Indiferentes,Insatisfechos,¿Factible?
Estadisticas,1.1756,615.96,71.05 %,25.26 %,3.69 %,True


,Tiempo (s),Puntaje SPLIT,Satisfechos,Indiferentes,Insatisfechos,#Conductores,#Conductores solución
Estadisticas,0.4695,615.96,71.05 %,25.26 %,3.69 %,163,163


Mejor solución asignación secuencial:
----------------------------------------------
0 Errores de secuencias prohibidas
867 de 867 turnos asignados
# Dias de trabajo  (min y max): 4 - 6 ( 3 - 6 )
# Dias de descanso (min y max): 1 - 3 ( 1 - 4 )
----------------------------------------------


,L,M,X,J,V,S,D
EMPLEADO 1,A,A,-,A,A,A,A
EMPLEADO 2,A,A,-,A,A,A,-
EMPLEADO 3,D,D,D,D,D,-,D
EMPLEADO 4,A,A,-,A,A,A,-
EMPLEADO 5,N,-,-,D,D,D,A
EMPLEADO 6,A,A,-,A,A,A,-
EMPLEADO 7,N,-,-,D,D,D,D
EMPLEADO 8,D,D,D,D,D,-,D
EMPLEADO 9,N,-,-,D,D,D,D
EMPLEADO 10,D,D,D,D,-,D,D


### **- Modelo Exacto**

In [13]:
modelo_exacto()

----------------------------------------------
Tiempo(s) = 1.283684492111206
Numero de turnos = 867
Puntaje = 656.9781
Satisfechos = 75.78%
Indiferentes = 22.03%
Insatisfechos = 2.19%
----------------------------------------------
0 Errores de secuencias prohibidas
867.0 turnos asignados
# Dias de trabajo (min y max): 3 - 6 ( 3 - 6 )
# Dias de descanso (min y max): 1 - 4 ( 1 - 4 )
----------------------------------------------


,L,M,X,J,V,S,D
EMPLEADO 1,A,A,A,-,A,A,A
EMPLEADO 2,A,A,A,A,A,-,A
EMPLEADO 3,D,D,D,D,D,D,-
EMPLEADO 4,A,A,A,A,-,A,A
EMPLEADO 5,-,N,-,-,-,D,A
EMPLEADO 6,A,-,A,A,A,A,A
EMPLEADO 7,D,D,D,D,-,D,D
EMPLEADO 8,D,-,D,D,D,D,D
EMPLEADO 9,D,-,D,D,D,D,D
EMPLEADO 10,D,-,D,D,D,D,D
